In [1]:
%load_ext watermark
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns
from myst_nb import glue
from slugify import slugify

from plastockconf import name_zones, name_frequentation, name_situation
from plastockconf import name_substrate, name_distance, table_css_styles, table_css_styles_top

from plastock import add_table_to_page, capitalize_x_tick_labels, capitalize_x_and_y_axis_labels, capitalize_legend_components, attribute_summary

import reportclass as rc
import setvariables as conf_


import matplotlib as mpl


format_kwargs = dict(precision=2, thousands="'", decimal=",")
def make_exportable(data, file_name, cmap='YlOrBr'):
    data.fillna(0, inplace=True)
    fig, ax = plt.subplots(figsize=(12,8))
    sns.heatmap(data=data, vmin=0, vmax=1, cmap=cmap, annot=True, fmt='.2', annot_kws={'size':10}, ax=ax, cbar=False)
    plt.tight_layout()
    ax.tick_params(which='both', axis='both', bottom=False, left=False)
    plt.savefig(file_name, dpi=300)

    plt.close()

glue('blank_caption', " ", display=False)

In [2]:
new_data = pd.read_csv("data/end_pipe/macro_current.csv")
beach_data = pd.read_csv("data/pstock_beaches_current.csv")
codes = pd.read_csv('data/end_pipe/codes.csv').set_index('code')

new_column_names = {
    "Position":"position",
    "Substrat":"substrat",
    "Date":"date",
    "Code":"code",
    "Quantité":"quantité",
    "Aire":"area"
}

length_key = beach_data[["Plage","length"]].drop_duplicates("Plage").set_index("Plage")
work_data = new_data[["Plage", *new_column_names.keys()]].copy()
work_data.rename(columns=new_column_names, inplace=True)
work_data["length"] = work_data.Plage.apply(lambda x: length_key.loc[x, "length"])
work_data["slug"] = work_data.Plage.apply(lambda x: slugify(x))
work_data["echantillon"] = list(zip(work_data.slug, work_data['date']))
work_data['date'] = pd.to_datetime(work_data["date"], format="mixed", dayfirst=True)
work_data.dropna(inplace=True)
work_data[["position", "substrat"]] = work_data[["position", "substrat"]].astype("int")
work_data['échantillon'] = work_data['echantillon']
work_data.drop(['echantillon'], inplace=True, axis=1)
work_data = work_data.groupby(['échantillon', 'Plage', 'substrat', 'date', 'length', 'slug', 'code'], as_index=False).agg({'quantité':'sum'})
work_data['pcs/m'] = work_data['quantité']/work_data['length']


voi = 'substrat'
vals = "pcs/m"
some_data = work_data.copy()
groupby = ['échantillon', voi]
data = some_data.groupby(groupby, as_index=False)[vals].sum()

# these are the duplicate values that need to be changed
dd = data[data['échantillon'].duplicated()].copy()

# duplicates = work_data[work_data['échantillon'].isin(ddd)].copy()
duplicated = work_data[work_data['échantillon'].isin(dd['échantillon'].unique())].copy()
duplicated['substrat'] = 2 

# notduplicated
not_duplicated = work_data[~(work_data['échantillon'].isin(dd['échantillon'].unique()))].copy()

# put it back to gether again
work_data = pd.concat([duplicated, not_duplicated])
work_data = work_data.groupby(['échantillon', 'Plage', 'substrat', 'date', 'length', 'slug', 'code'], as_index=False).agg({'quantité':'sum'})
work_data['pcs/m'] = work_data['quantité']/work_data['length']

# accounting for objects not found at a sample:
# the codes that were indentified
codes_ip = work_data.code.unique()
# the unique samples
loc_dates = work_data['échantillon'].unique()

# a copy for itterating
wd = work_data.copy()

# for each sample (échantillon) indentify the codes that were not
# found by indentifying all the codes that were found in all surveys
# and removing the codes that were not identified at that sample.
# for each unidentified code per sample, add a row with the sample
# id and the code. give the row a quantity of zero.
rows = []
for a_loc in loc_dates:
    r = wd.loc[wd['échantillon'] == a_loc].copy()
    r.reset_index(inplace=True, drop=True)
    
    t = r.loc[0][['échantillon', 'Plage', 'substrat', 'date', 'length', 'slug']].values
    asamp = [x for x in t]
    used_codes = r.code.unique()
    unused = [x for x in codes_ip if x not in used_codes]
    for element in unused:
        arow = [*asamp, element, 0, 0]
        rows.append(arow)
        

work_x = pd.DataFrame(rows, columns=['échantillon', 'Plage', 'substrat', 'date', 'length', 'slug', 'code', 'quantité', 'pcs/m'])
work_data = pd.concat([work_x, work_data])

# Résultats précédents

Les déchets sauvages sur les plages du Lac Léman font l'objet d'une surveillance depuis 2015. La campagne la plus récente avant Plastock a été parrainée par l'Office fédéral de l'environnement (OFEV) de 2020 à 2021. Les directives européennes sur la surveillance des déchets sauvages sur les plages ont été publiées en 2013 [Guidance on Monitoring Marine Litter](https://mcc.jrc.ec.europa.eu/documents/201702074014.pdf). Les méthodes utilisées pour surveiller les déchets sauvages sur le lac Léman suivent les directives de l'UE, tout en tenant compte des contraintes liées aux différences de topographie entre le littoral maritime et le littoral lacustre.

## Le système de mesure.

Les déchets sauvages sur les plages sont mesurés en unités de déchets par mètre linéaire de rivage. Les relevés sont effectués entre la ligne d'eau (là où l'eau et la terre se rejoignent) et la ligne de dérive (la distance la plus éloignée de la ligne d'eau atteinte par les vagues). Cette méthode a été adoptée dès le début en Suisse, ce qui signifie que les 250 échantillons précédents du lac Léman ont été collectés et mesurés selon la méthode précédemment décrite.

__Dans cette section, les unités sont des déchets par mètre de rivage: (pcs/m)__

## Resumé des résultats

La médiane des déchets par mètre (pcs/m) pour l'ensemble des 98 échantillons était de 2, soit 200 déchets pour 100 mètres. Cependant, selon l'urbanisation de la zone environnante, les taux d'utilisation estimés et le type de substrat, les taux attendus peuvent varier. Nous avons constaté que les lieux avec une plage de sable avaient la valeur médiane la plus élevée, soit 5,24 pcs/m pour 27 échantillons, suivis de près par les plages avec les taux d'utilisation les plus élevés, soit 3,4 pcs/m pour 55 échantillons. Les lieux classés comme urbains avaient une valeur médiane de 2,89 pour 31 échantillons, contre 1,63 pcs/m pour les lieux en campagne (67 échantillons). [Conditions](sampling_conditions)

Par rapport aux résultats de la dernière campagne d'échantillonnage sur le lac, 2020 - 2021 et 98 échantillons [Léman 2021](https://hammerdirt-analyst.github.io/IQAASL-End-0f-Sampling-2021/lac-leman.html), ceci représente une réduction de 50 % dans la médiane d'échantillonnage. Cette réduction est conforme aux conclusions de 2020 - 2021, bien qu'elle soit plus importante que prévu [SLR - IQAASL](https://www.plagespropres.ch/slr_2017.html#conclusion). La plus forte diminution concernait les objets associés à la consommation personnelle sur la plage (mégots de cigarettes, emballages de snacks et bouchons de bouteilles en plastique). La baisse marquée de la densité des cotons-tiges en plastique (0,13 pcs/m contre 0,03 pcs/m) était inattendue et pourrait être attribuée à l'interdiction de la vente de ces articles en France. [Plus courants Plastock](most_common_p), [Interdiction France](https://www.economie.gouv.fr/cedef/interdiction-plastique-usage-unique)

Il est à noter que la densité des plastiques fragmentés (Gfrags) a légèrement augmenté (0,68 pcs/m contre 0,72) et était également anticipée dans le rapport fédéral de 2021. La catégorie des Gfrags est une catégorie fourre-tout pour les objets en plastique qui ne peuvent pas être identifiés positivement comme appartenant à une catégorie de produit spécifique. Les bénévoles qui ne sont pas sûrs de l'utilisation d'un objet classifient l'objet comme Gfrag. Cette seule catégorie représentait 41 % du nombre total d'objets trouvés en 2022 contre 15 % en 2021. Considérant qu'il y a également eu des baisses significatives dans les plastiques de construction et les feuilles industrielles en 2022, il est probable que des objets qui auraient été classifiés comme plastiques de construction ou feuilles industrielles aient été classifiés comme Gfrags. [Plus courants de Plastock-IQAASL](combined_common)

__OSPAR :__ Parmi les 15 objets les plus courants dans le dernier rapport OSPAR, 6 sont également parmi les plus courants sur le Léman. [Correspondances ospar](cor_ospar)

### Différences entre les types de plage

__Haute fréquentation ville/campagne__

Les deux catégories ville et campagne combinées représentent 100 % de tous les échantillons. Rappelons que les lieux où l'on s'attend à un taux d'utilisation élevé représentent 56 % de tous les échantillons. À taux d'utilisation constant, l'échantillon médian attendu est plus élevé dans les localités désignées comme étant à la campagne (4,59 pcs/m contre 3,27 pcs/m). Cependant, à la campagne, on s'attend à trouver plus de Gfrags et de pellets industriels en % du total. Dans les villes, nous nous attendons à trouver plus de bouts de cigarettes et d'emballages de snacks par échantillon. [Random forest](random_forest)

__Distance entre le parking et la ville ou la campagne__

La distance jusqu'au parking est une mesure de l'accessibilité. En maintenant la distance constante et en tenant compte de l'urbanisation de l'emplacement, les emplacements urbains ont une valeur minimale et maximale attendue plus élevée ainsi qu'un pourcentage plus élevé de mégots de cigarettes. [Random forest](random_forest)

\*Le resumé des prédictions en pcs/m attendus en fonction de l'usage, la situation et le substrat :


::::{grid} 1

:::{grid-item}

|Situation|5%|25%|50%|75%|95%|
|:-:|:-:|:-:|:-:|:-:|:-:|
|Tous les variables |0.44|1.37|3.09|4.35|6.25|

_Fourchette prévue de valeurs d'échantillonnage pour tous les variables_
:::

:::{grid-item}

|Situation|5%|25%|50%|75%|95%|
|:-:|:-:|:-:|:-:|:-:|:-:|
|Ville|2.44|3.45|4.02|4.62|5.63|
|Campagne|2.24|3.24|3.89|4.67|5.89|

_Fourchette prévue de valeurs d'échantillonnage pour les lieux à forte fréquentation_ [sites concernés](freq_plages)
:::

:::{grid-item}

|Situation|5%|25%|50%|75%|95%|
|:-:|:-:|:-:|:-:|:-:|:-:|
|Sabloneuse|1.32|1.86|2.32|3.15|5.19|
|Graviers|0.35|0.81|3.97|5.48|7.38|

Fourchette prévue de valeurs d'échantillonnage pour les substrats diffèrents 
:::

:::{grid-item}

|Situation|5%|25%|50%|75%|95%|
|:-:|:-:|:-:|:-:|:-:|:-:|
|Ville|2.4|3.46|3.99|4.65|5.58|
|Campagne|1.43|2.09|3.15|4.55|6.55|

_Fourchette prévue de valeurs d'échantillonnage pour les emplacements situés à moins de 500 m du parking_ [sites concernés](dist_plage)
:::

::::

\*Ces catégories représentent 100% des échantillons.

### Exigences particulières données plastock

La quantité de déchets sauvages par mètre de plage correspond au nombre total d'objets identifiés divisé par la longueur du rivage. Pour les données Plastock, cela signifie que nous devons considérer la position un (ligne d'eau) et la position deux (plage sèche) ensemble. De plus, il y a __douze échantillons où le substrat était différent entre la position un et la position deux__. Par conséquent, ces 12 échantillons ont tous été classés dans la catégorie Sables grossiers.

#### La distribution de la variable de substrat après la réattribution des 12 échantillons en question.

1. Sables fins : 27%
2. Sables Grossiers : 32%
3. Graviers : 16%
4. Cailloux : 25%

In [3]:
# !important formatting data for use with IQAASL
# when combined with previous data there are duplicate values
# the locations in plastock data that have the same name as iqaasl data
# need to be changed
change_names = ['preverenges', 'tolochenaz', 'versoix', 'vidy', 'cully']

plastock_cols = ['loc_date', 'date','slug', 'code', 'quantity', 'city', 'feature_name', 'feature_type','parent_boundary', 'pcs_m']
features = ['frequentation', 'situation', 'orientation', 'distance']

changeus = work_data[work_data.slug.isin(change_names)].copy()
donotchange = work_data[~work_data.slug.isin(change_names)].copy()

new_slug = {
    'cully': 'cully-p',
    'preverenges': 'preverenges-p',
    'tolochenaz': 'tolochenaz-p',
    'versoix':'versoix-p',
    'vidy': 'vidy-p'}

# they have the same name as locations in iqaasl
changeus['new_slug'] = changeus.slug.apply(lambda x: new_slug[x])
changeus['slug'] = changeus.new_slug
changeus.drop('new_slug', inplace=True, axis=1)

# the plastock data with the converted names
wd_nn = pd.concat([changeus, donotchange])

# plastock did not use the same inventory as iqaasl
# here we select only the codes in the plastock inventory
pcodes = wd_nn.code.unique()

# identify and remove codes for which there is no defintion
# if the code is not defined then it can not be used
t = [x for x in pcodes if x not in codes.index]
wd_ni = wd_nn[~wd_nn.code.isin(t)].copy()

# these items are not well divided into the composite subgroups
# for example people often know what a cap is, but whether it 
# comes from a drink bottle or other type is not well considered
# we combine the subcategories into more comprehensive groups.
ti = rc.use_gfrags_gfoams_gcaps(wd_ni, codes)

# aggregate along all land-use and topo variables.
ti = ti.groupby(['échantillon', 'Plage', 'date', 'substrat', 'length', 'slug', 'code'], as_index=False).agg({'quantité':'sum'})

# the independent variables are in asl_beaches file
beach_data = pd.read_csv("data/end_pipe/asl_beaches.csv").set_index('Plage')

# !combinining with previous results!
# these are the default arguments for the report class
# the language maps gives the code definitions in english, german and french
# the top_label asserts the top level aggregation for the set of data defined by
# start, end dates and feature_name. These arguments are for the plastock data
language_maps = rc.language_maps()
top_label= ['feature_name', 'lac-leman']

# the default language is english in the report column class
# there are column names that need to be changed
new_names = {'échantillon': 'loc_date', 'pcs/m': 'pcs_m'}
ti.rename(columns={**new_names,'quantité': 'quantity'}, inplace=True)

# define the pcs/m column and the data to merge
ti['pcs_m'] = ti.quantity/ti.length

# the city designation is used for reporting
# the locations have been integrated with the iqaasl data for other models
# we can grab the city and other classifiers from the current set to of data 
f = pd.read_csv('data/u_pstk.csv')
city_map = f[["slug", "city"]].drop_duplicates()
city_map.loc[city_map.slug == 'savoniere', 'slug'] = 'savonniere'
city_map.set_index('slug', inplace=True)

# adding and renaming columns according to reportclass requirements
# these values can be indexed on the IQAASL data
ti['city'] = ti.slug.apply(lambda x: city_map.loc[x])
ti['feature_name'] = 'lac-leman'
ti['feature_type'] = 'l'
ti['parent_boundary'] = 'rhone'

# they can be merged on the Plage column and the index
env_plastock = ti.merge(beach_data[features], left_on='Plage', right_index=True)
env_plastock = env_plastock[[*plastock_cols, *features, 'substrat']]

ti_work = ti[plastock_cols].copy()

# this data is formatted to work with the reporting structure of IQAASL
# the landuse data is not included here.
ti_work = ti_work.groupby(plastock_cols, as_index=False).agg(conf_.unit_agg)
ti_work['project']='Plastock'

# previous results
ghi = pd.read_csv('data/end_pipe/iqaasl.csv')

# merge the data and select only the current codes from plastock
txi = pd.concat([ghi, ti_work[[*plastock_cols, 'project']].copy()])
txi.reset_index(inplace=True)
txi = txi[txi.code.isin(ti_work.code.unique())]

# a report that includes both sets of data
boundaries = dict(start_date="2020-04-01", end_date="2023-01-01", feature_name="lac-leman", language="fr")
report_iq_pk = rc.ReportClass(txi.copy(), boundaries=boundaries, language="fr", lang_maps=language_maps, top_label=top_label)

# a report that includes just plastock data
boundaries = dict(start_date="2021-12-31", end_date="2023-01-01", feature_name="lac-leman", language="fr")
plastock_report = rc.ReportClass(ti_work.copy(), boundaries=boundaries, language="fr", lang_maps=language_maps, top_label=top_label)

header = plastock_report.a_short_description.fillna('.')
comp_summary = plastock_report.summarize_feature_labels(feature='feature_name')
most_common, weight = plastock_report.most_common

In [4]:
# the changes in G27 in relation to the different land use variables.
operations =  {'échantillon':'nunique', 'pcs_m':'median'}

def make_categorical_matrix(data: pd.DataFrame = None, feature_columns: list = None, operations: dict = operations):    
    # executes a pd.DataFrame.groupby operation on data feature_columns using operations  
    nd = data.groupby(feature_columns, as_index=False).agg(operations)    
    
    return nd


def name_the_new_distance(x, less='<= 500 m', more = '> 500 m'):
    if x == 1:
        return less
    else:
        return more

def name_the_new_freq(x, new):
    if x <= 2:
        return new
    else:
        return 'Elévée'
    
    

# apply to the survey data
t_and_f = env_plastock.loc[:, ['loc_date', 'slug','date','code', 'pcs_m', 'quantity', 'frequentation', 'situation', 'distance', 'substrat']].copy()

# creation of composite variables
# the substrat and distance features are being combined
# the two lowest and the two highest of each group are being combined
# substrat is a matter of combining different granularities. They are being grouped as
# sand and gravel.
# distance is now grouped by locations either less than or equal to 500 meters
t_and_f.loc[t_and_f.substrat <= 2, 'substrat'] = 1
t_and_f.loc[t_and_f.substrat > 2, 'substrat'] = 2
t_and_f.loc[t_and_f.distance <= 2, 'distance'] = 1
t_and_f.loc[t_and_f.distance > 2, 'distance'] = 2
t_and_f.loc[t_and_f.frequentation <= 2, 'frequentation'] = 2

f_combi = t_and_f.copy()

f_combi.rename(columns={'frequentation':'fréquentation', 'loc_date': 'échantillon'}, inplace=True)

mask = (f_combi.code == 'G27')
f_comb = f_combi.copy()
f_comb['distance'] = f_comb['distance'].apply(lambda x: name_the_new_distance(x))
f_comb['fréquentation'] = f_comb['fréquentation'].apply(lambda x: name_the_new_freq(x, 'faible-moyenne'))
f_comb['situation'] = f_comb['situation'].apply(lambda x: name_situation[x])
f_comb['substrat'] = f_comb['substrat'].apply(lambda x: name_the_new_distance(x, less='Sable', more='Graviers'))

f_freq_sub = make_categorical_matrix(data=f_comb, feature_columns=['fréquentation', 'substrat'])
f_freq_sit = make_categorical_matrix(data=f_comb, feature_columns=['fréquentation', 'situation'])
f_freq_dist = make_categorical_matrix(data=f_comb, feature_columns=['fréquentation', 'distance'])



# The work data for the GPT. The data of reference.
# We will add the orientation column later.
no_combined = env_plastock.loc[:, ['loc_date', 'slug','date','code', 'pcs_m', 'frequentation', 'situation', 'distance', 'substrat']].copy()
no_combined.rename(columns={'frequentation':'fréquentation', 'loc_date': 'échantillon'}, inplace=True)
# for the GPT:
no_combined.to_csv('plastock_with _asl_landuse.csv', index=False)

operations =  {'échantillon':'nunique', 'pcs_m':'median'}

no_combined['distance'] = no_combined['distance'].apply(lambda x: name_distance[x])
no_combined['fréquentation'] = no_combined['fréquentation'].apply(lambda x: name_frequentation[x])
no_combined['situation'] = no_combined['situation'].apply(lambda x: name_situation[x])
no_combined['substrat'] = no_combined['substrat'].apply(lambda x: name_substrate[x])

In [5]:
def check_dfs(df, label, samps='echanitllon', pcs='pcs_m'):
    dts = df.groupby([samps, 'slug'], as_index=False)[pcs].sum()
    quants = dts[pcs].quantile([.05, .25, .5, .75, .95])
    sampls =dts[samps].nunique()
    
    # print(f'\n{label}: \n{quants}\n{sampls}')
    
    return quants, sampls

nc_q, nc_samps = check_dfs(no_combined, 'not combined',  samps='échantillon')
fcmb_q, fcmb_samps = check_dfs(f_combi, 'combined', samps='échantillon')
tf_q, tf_samps = check_dfs(t_and_f,'feature_t', samps='loc_date')
ti_q, ti_samps = check_dfs(ti_work,'report data', samps='loc_date')
trc, trc_s = check_dfs(plastock_report.w_df, 'report df', samps='loc_date')

## Situation

Les déchets sauvages ont été collectés dans 25 emplacements différents autour du Léman. Soixante-sept sur 98 de ces emplacements se trouvaient dans des zones urbaines, avec un taux d'utilisation élevé de 55 sur 98, et 83 sur 98 étaient situés à moins de 500 mètres d'un parking. Les caractéristiques ont été notées par les personnes chargées de la collecte des déchets sauvages [Macro déchets plage et attribut](macro-attributes).

### Les résultats

In [6]:
t = rc.translated_and_style_for_display(comp_summary.copy(),  plastock_report.lang_maps[plastock_report.language], plastock_report.language, gradient=False)
rep_res_one = "L'échantillonnage était planifié trimestriellement, commençant en janvier 2022. À quelques exceptions près, chaque emplacement était échantillonné quatre fois au cours de la période de 12 mois. Les études précédentes sur le lac étaient mensuelles ou aléatoires."
rep_res_two = "La médiane du total des échantillons était de 2 pièces/mètre avec une variance de 3,6 pièces/mètre."

glue('rep_results_one',rep_res_one , display=False)
glue('rep_results_two',rep_res_two , display=False)
glue('rep_results', t, display=False)

<style>
table th:first-of-type {
    width: 50%;
}
table th:nth-of-type(2) {
    width: 50%;
}

</style>


| |La répartition des résultats|
|:---:|:---|
|{glue}`rep_results`|<br>{glue:text}`rep_results_two`<br><br>{glue:text}`rep_results_one`|

In [7]:
comb_dt = plastock_report.w_df
comb_dt = comb_dt.groupby(['loc_date', 'date', 'project'], as_index=False).pcs_m.sum()

fig, ax = plt.subplots(2,2, figsize=(8,8))

sns.scatterplot(data=comb_dt, x="date", y="pcs_m", hue='project', ax=ax[0,0])
ax[0,0].set_title("Total par échantillon", loc="left")
ax[0,0].xaxis.set_major_locator(mdates.MonthLocator(interval=3))
ax[0,0].xaxis.set_major_formatter(mdates.DateFormatter('%m-%y'))
ax[0,0].set_ylabel("pcs/m")
capitalize_x_and_y_axis_labels(ax[0,0])
capitalize_legend_components(ax[0, 0])

sns.boxplot(data=comb_dt, y="pcs_m", x='project', hue='project', dodge=False, width=.9, ax=ax[0,1])
ax[0,1].set_title("Boîte de Tukey", loc="left")
capitalize_x_and_y_axis_labels(ax[0,1])
ax[0,1].get_legend().remove()
ax[0,1].set_xlabel("")
ax[0,1].set_ylabel("pcs/m")

sns.histplot(data=comb_dt, x="pcs_m", ax=ax[1,0], stat="probability", hue='project', kde=True)
ax[1,0].yaxis.set_major_formatter('{x:.2f}')
ax[1,0].set_ylabel("Probabilité")
ax[1,0].set_xlabel("pcs/m")
ax[1,0].set_title("Histogramme", loc="left")
capitalize_x_and_y_axis_labels(ax[1,0])
ax[1, 0].get_legend().remove()

sns.ecdfplot(data=comb_dt, x="pcs_m", hue='project', ax=ax[1,1])
ax[1,1].set_title("Fonction de répartition", loc="left")
ax[1,1].set_xlabel("pcs/m")
capitalize_x_and_y_axis_labels(ax[1,1])
sns.move_legend(ax[1, 1], title=" ", loc='best')

plt.subplots_adjust(wspace=.3, hspace=.3)

glue('fig-D1', fig, display=False)
plt.close()


```{glue:figure} fig-D1
---
name: fig-D1
---
{glue:text}`blank_caption` 
```

(sampling_conditions)=
## Les conditions d'échantillonnage

Les enquêteurs ont classé les conditions à chaque emplacement d'échantillonnage selon quatre catégories. :

1. distance au parking, du plus proche au plus éloigné (1-4)
2. utilisation : faible, moyenne, élevée
3. substrat : sable fin à roches (1-4)
4. situation : campagne ou urbain

In [8]:
def calculate_combined_stats(category, data=no_combined.copy()):
    # Descriptive statistics of the sample density and quantity of pieces found
    # Aggregates the data on category and sample_id
    
    grouped = data.groupby([category, 'échantillon'], as_index=False).pcs_m.sum()

    group_summary = grouped.groupby(category, as_index=True).agg({'échantillon':'nunique', 'pcs_m':['mean', 'median']})
    group_summary.columns = group_summary.columns.droplevel(0)
    # Calculating percentage of total samples
    group_summary['percentage'] = (group_summary['nunique'] / 98) * 100
    group_summary.reset_index(drop=False, inplace=True)

    # Renaming columns for clarity
    group_summary.rename(columns={'nunique': 'échantillons','mean': 'Moyenne', 'median': 'Médiane', 'percentage': '% du total'}, inplace=True)
    return group_summary

# Calculate stats for each category with combined rows
frequentation_stats = calculate_combined_stats('fréquentation')
situation_stats = calculate_combined_stats('situation')
distance_stats = calculate_combined_stats('distance')
substrat_stats = calculate_combined_stats('substrat')

frequentation_stats.style.set_table_styles(table_css_styles).format(precision=2)

,fréquentation,échantillons,Moyenne,Médiane,% du total
0,Elevée,55,4.16,3.40,56.12
1,Faible,15,0.59,0.44,15.31
2,Moyenne,28,3.49,1.63,28.57


In [9]:
situation_stats.style.set_table_styles(table_css_styles).format(precision=2)

,situation,échantillons,Moyenne,Médiane,% du total
0,Campagne,67,3.46,1.63,68.37
1,Urbain,31,3.33,2.89,31.63


In [10]:
distance_stats.style.set_table_styles(table_css_styles).format(precision=2)

,distance,échantillons,Moyenne,Médiane,% du total
0,100 - 500 m,44,4.31,2.95,44.90
1,500 - 1000 m,7,0.63,0.43,7.14
2,< 100 m,39,3.47,2.83,39.80
3,> 1000 m,8,0.72,0.71,8.16


In [11]:
substrat_stats.style.set_table_styles(table_css_styles).format(precision=2)

,substrat,échantillons,Moyenne,Médiane,% du total
0,Cailloux,24,2.63,1.76,24.49
1,Graviers,16,1.68,1.08,16.33
2,Sables fins,27,6.74,5.24,27.55
3,Sables grossiers,31,2.04,1.48,31.63


### Résumé comparatif :

1. Dans l'ensemble global, les plages très fréquentées (fréquentation 3) représentent plus de la moitié des échantillons. Cela peut influencer les résultats globaux.
2. Pour les plages très fréquentées, les moyennes pcs/m sont généralement plus élevées que dans l'ensemble global, indiquant une plus grande accumulation de débris.
3. La répartition entre les zones urbaines et rurales (situation) est presque égale parmi les plages très fréquentées, alors qu'elle est plus inclinée vers les zones rurales dans l'ensemble global.
4. En termes de substrat, les plages de sable sont les plus représentées dans les échantillons très fréquentés, ce qui est également observé dans l'ensemble global.

Ces différences mettent en évidence l'impact significatif de la fréquentation élevée sur la pollution des plages

#### Détail fréquentation élevée:

In [12]:
freq_data = no_combined[no_combined['fréquentation'] == 'Elevée'].copy()

freq_sit = calculate_combined_stats('situation', data=freq_data)

freq_dist = calculate_combined_stats('distance', data=freq_data)

freq_subs = calculate_combined_stats('substrat', data=freq_data)

freq_sit.style.set_table_styles(table_css_styles).format(precision=2)

,situation,échantillons,Moyenne,Médiane,% du total
0,Campagne,28,4.60,3.55,28.57
1,Urbain,27,3.69,3.40,27.55


In [13]:
freq_dist.style.set_table_styles(table_css_styles).format(precision=2)

,distance,échantillons,Moyenne,Médiane,% du total
0,100 - 500 m,36,3.50,2.80,36.73
1,< 100 m,19,5.40,4.42,19.39


In [14]:
freq_subs.style.set_table_styles(table_css_styles).format(precision=2)

,substrat,échantillons,Moyenne,Médiane,% du total
0,Cailloux,8,3.76,3.19,8.16
1,Graviers,8,2.91,1.57,8.16
2,Sables fins,23,5.54,4.83,23.47
3,Sables grossiers,16,3.00,2.56,16.33


## Les objets trouvés en fonction de leur utilisation

Le type d'utilité est basé sur l'utilisation de l'objet avant qu'il ne soit jeté ou sur la description de l'objet si l'utilisation initiale est indéterminée. Les objets identifiés sont classés dans l'une des 260 catégories prédéfinies. Les catégories sont regroupées en fonction de leur utilisation ou de leur description.

- Eaux usées : objets rejetés par les stations d'épuration, y compris les objets susceptibles d'être jetés dans les toilettes.
- Microplastiques (< 5 mm) : plastiques fragmentés et résines plastiques de préproduction.
- Infrastructure : objets liés à la construction et à l'entretien des bâtiments, des routes et des réseaux d'eau et d'électricité.
- Alimentation et boisson : tous les matériaux liés à la consommation de nourriture et de boissons.
- Agriculture : principalement des feuilles industrielles, par exemple, paillis et bâches de culture, serres, fumigation du sol, films d'emballage de balles. Comprend les plastiques durs pour les clôtures agricoles, les pots de fleurs, etc.
- Tabac : principalement des filtres de cigarettes, y compris tous les matériaux liés au tabagisme.
- Loisirs : objets liés au sport et aux loisirs, par exemple, pêche, chasse, randonnée, etc.
- Emballages non alimentaires et non liés au tabac : matériaux d'emballage non identifiés comme étant liés à la nourriture, aux boissons ou au tabac.
- Fragments de plastique : morceaux de plastique d'origine ou d'utilisation indéterminée.
- Objets personnels : accessoires, articles d'hygiène et vêtements.

Pour des informations détaillées sur la composition des groupes, consultez [IQAASL - DE](https://hammerdirt-analyst.github.io/IQAASL-End-0f-Sampling-2021/code_groups.html) ou [IQAASL - EN](https://www.plagespropres.ch/code_groups.html).
<br>

In [15]:
w_df = plastock_report.w_df.copy()
cities = w_df.city.unique()
cone = cities[:12]
ctwo = cities[12:]

groups_df = plastock_report.w_df.copy()
group_name_map = codes['groupname']
groups_df['groupname'] = groups_df.code.apply(lambda x: group_name_map.loc[x])
tg1 = rc.a_cumulative_report(groups_df, feature_name='city', object_column='groupname', table_split=cone)
rc.translated_and_style_for_display(tg1, plastock_report.lang_maps[plastock_report.language], plastock_report.language, gradient=True).set_caption("Les résultats des objets par utilisation pour chaque ville du projet: Amphion à Hemance")

,Amphion,Anthy,Aubonne,Bouveret,Clarens,Crans,Cully,Excenevex,Genève,Gland,Grangettes,Hermance,Cumulé
Agriculture,"0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,01","0,00","0,00"
Nourriture et boissons,"0,70","0,01","0,17","0,84","0,31","0,05","0,00","0,21","0,20","0,02","2,02","0,04","0,22"
Infrastructures,"0,51","0,01","0,07","0,92","0,64","0,00","0,00","0,10","0,10","0,65","2,54","0,02","0,11"
Micro-plastiques (< 5mm),"0,41","0,01","0,15","1,46","0,05","0,00","0,06","2,16","1,26","0,09","1,99","0,00","0,10"
Emballage non alimentaire,"0,75","0,00","0,07","0,41","0,05","0,00","0,06","0,06","0,07","0,00","0,33","0,02","0,04"
Articles personnels,"0,07","0,02","0,00","0,05","0,04","0,00","0,03","0,02","0,14","0,00","0,11","0,02","0,03"
Morceaux de plastique,"2,82","0,30","0,24","2,18","1,34","0,14","0,14","5,17","0,84","0,63","5,79","0,25","0,72"
Loisirs,"0,17","0,02","0,03","0,14","0,05","0,00","0,06","0,06","0,14","0,00","0,17","0,00","0,05"
Tabac,"0,13","0,06","0,19","0,43","1,19","0,00","0,06","0,37","0,36","0,11","0,34","0,12","0,15"
Non classé,"0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00"


In [16]:
file_name = 'resources/images/group_names_one.jpg'
make_exportable(tg1, file_name)

In [17]:
tg2 = rc.a_cumulative_report(groups_df, feature_name='city', object_column='groupname', table_split=ctwo)
rc.translated_and_style_for_display(tg2, plastock_report.lang_maps[plastock_report.language], plastock_report.language, gradient=True).set_caption("Les résultats des objets par utilisation pour chaque ville du projet: Lugirn à Vevey")

,Lugrin,Lutry,Meillerie,Préverenges,Rolle,Saint-disdille,Savonière,Tolochenaz,Tougues,Versoix,Vevey,Vidy,Cumulé
Agriculture,"0,03","0,00","0,00","0,00","0,00","0,32","0,00","0,00","0,00","0,00","0,00","0,00","0,00"
Nourriture et boissons,"0,36","0,32","0,00","0,28","0,21","0,79","0,43","0,10","0,13","0,30","0,06","0,84","0,22"
Infrastructures,"0,61","0,10","0,22","0,20","0,26","0,30","0,06","0,03","0,07","0,14","0,00","0,11","0,11"
Micro-plastiques (< 5mm),"0,00","0,02","0,14","1,10","0,87","0,07","0,02","0,06","0,08","0,00","0,04","0,39","0,10"
Emballage non alimentaire,"0,07","0,04","0,00","0,03","0,01","0,33","0,14","0,00","0,00","0,07","0,01","0,16","0,04"
Articles personnels,"0,09","0,00","0,00","0,04","0,13","0,08","0,06","0,01","0,01","0,02","0,03","0,04","0,03"
Morceaux de plastique,"0,49","1,72","0,18","2,40","1,15","1,95","0,78","0,21","0,31","1,05","0,05","1,63","0,72"
Loisirs,"0,09","0,06","0,00","0,07","0,10","0,11","0,09","0,00","0,01","0,03","0,01","0,15","0,05"
Tabac,"0,09","0,00","0,08","0,42","0,66","0,34","0,20","0,00","0,54","0,00","0,04","1,58","0,15"
Non classé,"0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00"


In [18]:
file_name = 'resources/images/group_names_two.jpg'
make_exportable(tg2, file_name)

(most_common_p)=
## Les objets les plus courants Plastock

### Définition des _objets les plus courants_

Les _objets les plus courants_ peuvent être sélectionnés de plusieurs manières. On peut également les appeler les _objets d'intérêt_. Dans le cadre de ce rapport, nous nous concentrons sur les objets qui représentent une proportion plus importante des résultats que les autres. Nous avons utilisé deux critères de sélection : i. la quantité, ii. le taux d'échec.

1. Quanité: Si un objet a une quantité totale qui le place dans les dix premiers, il est considéré comme 'courant'.
2. Taux d'échec: Si un objet a été trouvé dans au moins la moitié des échantillons, il est ÉGALEMENT considéré comme 'courant'.

Par conséquent, pour cette étude, les 'objets les plus courants' sont ceux qui se trouvent soit dans les dix premiers en termes de nombre total de pièces de déchets ET/OU qui ont été trouvés dans au moins 50% des enquêtes. Pour Plastock, les objets les plus courants représentent 89% du montant total ou 24'156/27'493.

```{note}
Les équipes peuvent avoir d'autres critères pour s'intéresser à l'occurrence d'un objet dans l'environnement. Il s'agit d'un sujet qui devrait être largement développé au sein des équipes. 
```

In [19]:
# pd.set_option('max_colwidth',50)
xi = rc.translated_and_style_for_display(most_common.copy(),  plastock_report.lang_maps[plastock_report.language], plastock_report.language, gradient=False)
xi.set_caption("Les objets les plus courants Plastock")
glue('mc_pstock', xi, display=False)

```{glue} mc_pstock
```

### Les plus courants par ville

La densité médiane des objets les plus courants n'est pas la même pour chaque emplacement/municipalité, ce qui suggère une répartition géographiquement inégale de ces objets.

Résultats de Amphion à Hermance :

In [20]:
w_df = plastock_report.w_df.copy()
cities = w_df.city.unique()

t = rc.a_cumulative_report(w_df[(w_df.code.isin(most_common.index))], feature_name='city', object_column='code', table_split=cone)

rc.translated_and_style_for_display(t, plastock_report.lang_maps[plastock_report.language], plastock_report.language, gradient=True).set_caption("Les résultats des objets les plus courants de Plastock pour chaque ville du projet: Amphion à Hemance")

,Amphion,Anthy,Aubonne,Bouveret,Clarens,Crans,Cully,Excenevex,Genève,Gland,Grangettes,Hermance,Cumulé
Fragments de plastique angulaires <5mm,"0,00","0,00","0,03","0,68","0,00","0,00","0,00","0,89","0,46","0,00","0,31","0,00","0,00"
Pellets industriels (gpi),"0,00","0,01","0,00","0,42","0,02","0,00","0,00","0,05","0,27","0,09","0,27","0,00","0,00"
Mégots et filtres à cigarettes,"0,13","0,06","0,19","0,43","1,19","0,00","0,06","0,36","0,36","0,11","0,34","0,12","0,15"
"Emballages de bonbons, de snacks","0,43","0,00","0,08","0,43","0,24","0,05","0,00","0,13","0,16","0,00","1,67","0,02","0,15"
Bâtonnets de sucette,"0,12","0,01","0,01","0,15","0,05","0,00","0,00","0,06","0,00","0,00","0,00","0,00","0,02"
Mousse de plastique pour l'isolation thermique,"0,02","0,00","0,00","0,02","0,31","0,00","0,00","0,00","0,01","0,00","0,68","0,00","0,00"
Déchets de construction en plastique,"0,00","0,00","0,00","0,49","0,02","0,00","0,00","0,00","0,01","0,00","0,24","0,00","0,00"
Coton-tige,"0,70","0,02","0,00","0,19","0,27","0,00","0,00","0,00","0,01","0,02","0,55","0,00","0,03"
"Couvercles en plastique bouteille: g21, g22, g23, g24","0,65","0,00","0,07","0,41","0,05","0,00","0,06","0,06","0,06","0,00","0,33","0,00","0,04"
"Fragments de polystyrène expansé: g81, g82, g83","0,45","0,01","0,03","0,30","0,21","0,00","0,00","0,09","0,04","0,48","1,31","0,00","0,07"


In [21]:
file_name = 'resources/images/most_common_one.jpg'
make_exportable(t, file_name)

<br>
<br>
Résultats de Lugrin à Vidy :

In [22]:
t = rc.a_cumulative_report(w_df[(w_df.code.isin(most_common.index))], feature_name='city', object_column='code', table_split=ctwo)

rc.translated_and_style_for_display(t, plastock_report.lang_maps[plastock_report.language], plastock_report.language, gradient=True).set_caption("Les résultats des objets les plus courants de Plastock pour chaque ville du projet: Lugirn à Vevey")

,Lugrin,Lutry,Meillerie,Préverenges,Rolle,Saint-disdille,Savonière,Tolochenaz,Tougues,Versoix,Vevey,Vidy,Cumulé
Fragments de plastique angulaires <5mm,"0,00","0,00","0,04","0,00","0,61","0,02","0,00","0,00","0,02","0,00","0,00","0,18","0,00"
Pellets industriels (gpi),"0,00","0,00","0,00","0,82","0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,15","0,00"
Mégots et filtres à cigarettes,"0,09","0,00","0,08","0,42","0,66","0,34","0,19","0,00","0,54","0,00","0,04","1,58","0,15"
"Emballages de bonbons, de snacks","0,31","0,05","0,00","0,26","0,21","0,62","0,29","0,10","0,10","0,20","0,05","0,65","0,15"
Bâtonnets de sucette,"0,02","0,25","0,00","0,01","0,01","0,08","0,07","0,01","0,02","0,00","0,00","0,06","0,02"
Mousse de plastique pour l'isolation thermique,"0,00","0,00","0,02","0,00","0,00","0,07","0,00","0,00","0,00","0,08","0,00","0,06","0,00"
Déchets de construction en plastique,"0,01","0,00","0,00","0,01","0,00","0,01","0,00","0,00","0,00","0,00","0,00","0,00","0,00"
Coton-tige,"0,14","0,00","0,02","0,06","0,00","0,23","0,15","0,00","0,00","0,15","0,00","0,30","0,03"
"Couvercles en plastique bouteille: g21, g22, g23, g24","0,07","0,04","0,00","0,03","0,01","0,31","0,14","0,00","0,00","0,07","0,01","0,16","0,04"
"Fragments de polystyrène expansé: g81, g82, g83","0,46","0,09","0,20","0,18","0,16","0,15","0,02","0,01","0,05","0,01","0,00","0,01","0,07"


In [23]:
file_name = 'resources/images/most_common_two.jpg'
make_exportable(t, file_name)

(cor_ospar)=
### Correspondance avec les résultats d'OSPAR

La Suisse et la France sont toutes deux parties contractantes de la convention [OSPAR](https://www.ospar.org/organisation/contracting-parties). Parmi les objets les plus courants identifiés à partir des données Plastock, six peuvent également être retrouvés dans le rapport d'état de qualité le plus récent d'OSPAR concernant l'Abondance, la Composition et les Tendances des [déchets sauvages](https://www.ospar.org/work-areas/eiha/marine-litter/assessment-of-marine-litter/beach-litter).

1. Fragments de plastique : g80, g79, g78, g77, g76, g75
2. Fragments de polystyrène expansé : g81, g82, g83
3. Couvercles en plastique de bouteille : g21, g22, g23, g24
4. Coton-tige
5. Emballages de bonbons, de snacks
6. Déchets de construction en plastique

```{note}
Les directives de surveillance d'OSPAR n'incluent pas les objets de moins de 5 mm. Par conséquent, des objets tels que les granulés industriels  (GPI) ou de petits morceaux de 5 mm ou moins ne figureront pas dans la liste des objets les plus courants d'OSPAR.

Selon le _Document de référence d'OSPAR sur les granulés plastiques pré-production_, [granulés OSPAR](https://www.ospar.org/documents?v=39764), entre 16 et 167 tonnes par an de GPI sont perdues dans l'environnement
```

### Correspondance avec l'IQAASL

Les objets les plus courants répertoriés pour le lac Léman en 2021 comprennent tous les objets les plus courants de Plastock à l'exception des 'Fragments de plastique angulaires <5 mm', [Lac Léman, IQAASL](https://hammerdirt-analyst.github.io/IQAASL-End-0f-Sampling-2021/).

```{note}
Les protections féminines et applicateurs de tampons, des éléments importants parmi les objets les plus courants d'OSPAR et d'IQAASL, n'ont pas été identifiées lors des enquêtes Plastock.
```

In [24]:
t = rc.a_cumulative_report(report_iq_pk.w_df[report_iq_pk.w_df.code.isin(most_common.index)], feature_name='project', object_column='code')
rc.translated_and_style_for_display(t, plastock_report.lang_maps[plastock_report.language], plastock_report.language, gradient=True).set_caption("Les objets les plus courants de Plastock et les résultats pour les mêmes objets de l'IQAASL")

,Iqaasl,Plastock,Cumulé
Fragments de plastique angulaires <5mm,"0,00","0,00","0,00"
Pellets industriels (gpi),"0,00","0,00","0,00"
Mégots et filtres à cigarettes,"0,47","0,15","0,30"
"Emballages de bonbons, de snacks","0,22","0,15","0,18"
Bâtonnets de sucette,"0,03","0,02","0,02"
Mousse de plastique pour l'isolation thermique,"0,10","0,00","0,02"
Déchets de construction en plastique,"0,06","0,00","0,00"
Coton-tige,"0,13","0,03","0,08"
"Couvercles en plastique bouteille: g21, g22, g23, g24","0,11","0,04","0,08"
"Fragments de polystyrène expansé: g81, g82, g83","0,24","0,07","0,11"


In [25]:
file_name = 'resources/images/most_common_iqaasl.jpg'
make_exportable(t, file_name)

(pstock_iqaasl)=
## Plastock - IQAASL

Plastock et IQAASL étaient deux projets distincts qui ont mesuré la même chose. En les combinant, ils offrent une représentation des conditions sur le littoral du lac pour la période allant d'avril 2020 à décembre 2022.

Dans cette section, nous comparons les résultats d'un projet à l'autre et les combinons pour définir les perspectives pour 2024.

In [26]:
rep_res_one = "Il y a eu 101 échantillons collectés dans 24 endroits différents de 13 villes différentes entre 2020 et 2021 sur le lac Léman. La médiane, la moyenne et l'écart type étaient tous plus élevés lors de l'exercice 2020-2021 par rapport à Plastock (2022)."
rep_res_two = "La longueur du littoral était en moyenne plus longue et plus large pour les enquêtes Plastock."
t = report_iq_pk.summarize_feature_labels(feature='project')
ti = rc.translated_and_style_for_display(t, plastock_report.lang_maps[plastock_report.language], plastock_report.language, gradient=False)
glue('pstock_iqaasl_one',rep_res_one , display=False)
glue('pstock_iqaasl_two',rep_res_two , display=False)
glue('mc_pstock_results', ti, display=False)

<style>
table th:first-of-type {
    width: 50%;
}
table th:nth-of-type(2) {
    width: 50%;
}

</style>


| |La répartition des résultats|
|:---:|:---|
|{glue}`mc_pstock_results`|<br>{glue:text}`pstock_iqaasl_one`<br><br>{glue:text}`pstock_iqaasl_two`|

In [27]:
comb_dt = report_iq_pk.w_df
comb_dt = comb_dt.groupby(['loc_date', 'date', 'project'], as_index=False).pcs_m.sum()

fig, ax = plt.subplots(2,2, figsize=(8,8))

sns.scatterplot(data=comb_dt, x="date", y="pcs_m", hue='project', ax=ax[0,0])
ax[0,0].set_title("Total par échantillon", loc="left")
ax[0,0].xaxis.set_major_locator(mdates.MonthLocator(interval=6))
ax[0,0].xaxis.set_major_formatter(mdates.DateFormatter('%m-%y'))
ax[0,0].set_ylabel("pcs/m")
capitalize_x_and_y_axis_labels(ax[0,0])
capitalize_legend_components(ax[0, 0])

sns.boxplot(data=comb_dt, y="pcs_m", x='project', hue='project', dodge=False, width=.9, ax=ax[0,1])
ax[0,1].set_title("Boîte de Tukey", loc="left")
capitalize_x_and_y_axis_labels(ax[0,1])
ax[0,1].get_legend().remove()
ax[0,1].set_xlabel("")
ax[0,1].set_ylabel("pcs/m")

sns.histplot(data=comb_dt, x="pcs_m", ax=ax[1,0], stat="probability", hue='project', kde=True)
ax[1,0].yaxis.set_major_formatter('{x:.2f}')
ax[1,0].set_ylabel("Probabilité")
ax[1,0].set_xlabel("pcs/m")
ax[1,0].set_title("Histogramme", loc="left")
capitalize_x_and_y_axis_labels(ax[1,0])
ax[1, 0].get_legend().remove()

sns.ecdfplot(data=comb_dt, x="pcs_m", hue='project', ax=ax[1,1])
ax[1,1].set_title("Fonction de répartition", loc="left")
capitalize_x_and_y_axis_labels(ax[1,1])
ax[1,1].set_xlabel("pcs/m")
sns.move_legend(ax[1, 1], title=" ", loc='best')

plt.subplots_adjust(wspace=.3, hspace=.3)

glue('fig-D2', fig, display=False)
plt.close()


```{glue:figure} fig-D2
---
name: fig-D2
---
{glue:text}`blank_caption` 
```

(combined_common)=
### Les objets les plus courants Plastock + Iqaasl

Les objets les plus courants des données combinées sont identifiés avec la même méthode que dans la section précédente. Les objets les plus courants ci-dessous représentent 87 % de l'ensemble des données, soit 46 003 sur 52 777.

In [28]:
com_mc, weight = report_iq_pk.most_common
caption = "Les objets les plus courants des données combinées de Plastock et IQAASL"
rc.translated_and_style_for_display(com_mc.copy(), plastock_report.lang_maps[plastock_report.language], plastock_report.language, gradient=False).set_caption(caption)

,Quantité,% du total,Pcs/m,Taux d'échec
"Fragments de plastique: g80, g79, g78, g77, g76, g75",15'996,"0,30","0,69","0,97"
Mégots et filtres à cigarettes,6'255,"0,12","0,30","0,87"
"Fragments de polystyrène expansé: g81, g82, g83",5'040,"0,10","0,11","0,78"
"Emballages de bonbons, de snacks",3'917,"0,07","0,18","0,85"
Pellets industriels (gpi),2'966,"0,06","0,00","0,42"
Fragments de plastique angulaires <5mm,2'357,"0,04","0,00","0,33"
"Couvercles en plastique bouteille: g21, g22, g23, g24",2'345,"0,04","0,08","0,77"
Coton-tige,2'234,"0,04","0,08","0,68"
Mousse de plastique pour l'isolation thermique,1'531,"0,03","0,02","0,58"
"Bâche, feuille plastique industrielle",1'279,"0,02","0,00","0,43"


#### Distribution des plus courants pour chaque projet.

In [29]:
t = rc.a_cumulative_report(report_iq_pk.w_df[report_iq_pk.w_df.code.isin(com_mc.index)], feature_name='project', object_column='code')
rc.translated_and_style_for_display(t, plastock_report.lang_maps[plastock_report.language], plastock_report.language, gradient=True).set_caption("Les résultats des objets les plus courants séparés par projet.")

,Iqaasl,Plastock,Cumulé
Fragments de plastique angulaires <5mm,"0,00","0,00","0,00"
Pellets industriels (gpi),"0,00","0,00","0,00"
Polystyrène < 5mm,"0,00","0,00","0,00"
Mégots et filtres à cigarettes,"0,47","0,15","0,30"
"Emballages de bonbons, de snacks","0,22","0,15","0,18"
Bâtonnets de sucette,"0,03","0,02","0,02"
Pailles et agitateurs,"0,04","0,00","0,02"
"Bâche, feuille plastique industrielle","0,09","0,00","0,00"
Mousse de plastique pour l'isolation thermique,"0,10","0,00","0,02"
Coton-tige,"0,13","0,03","0,08"


In [30]:
file_name = 'resources/images/most_common_combined.jpg'
make_exportable(t, file_name)

## Estimation des paramètres et prédictions





In [31]:
# check the ranked correlation betwen the ordinal variables
corr_table = env_plastock[env_plastock.code.isin(com_mc.index)].groupby(['loc_date', 'city', 'substrat','frequentation', 'situation', 'orientation', 'distance'], as_index=False).pcs_m.sum()
corr_table.rename(columns={'frequentation':'fréquentation'}, inplace=True)
corred = corr_table[['fréquentation', 'situation', 'distance', 'substrat']].corr(method='spearman')
caption = "La corrélation des variables indépendantes, à l'exception de l'orientation."
corred = rc.translated_and_style_for_display(corred, plastock_report.lang_maps[plastock_report.language], plastock_report.language, gradient=False).set_caption(caption)

glue('macro-correlation-one', corred, display=False)

In [32]:
operations = {'échantillon':'nunique'}
feature_columns = ['fréquentation']

data = make_categorical_matrix(no_combined.copy(), feature_columns=feature_columns, operations=operations)
data2 = no_combined.groupby(['fréquentation', 'distance'], as_index=False)['échantillon'].nunique()

In [33]:
fig, ax = plt.subplots(1, 2)

hue_order = ['Faible', 'Moyenne', 'Elevée']
ecdfd = no_combined.groupby(['échantillon', 'fréquentation', 'distance'], as_index=False).pcs_m.sum()

sns.barplot(data=data, x='fréquentation', y='échantillon', color='grey', ax=ax[0], label='fréquentation', order=hue_order, alpha=0.4, zorder=1)
sns.barplot(data=data2, x='fréquentation', y='échantillon', hue='distance',   order=hue_order, edgecolor='white', linewidth=2, palette='tab20', ax=ax[0], alpha=0.5, zorder=2)
caption = "L'utilisation de la plage et la distance jusqu'au parking"
sns.ecdfplot(data=ecdfd, x='pcs_m', ax=ax[1], hue='fréquentation', palette='tab20', hue_order=hue_order)
ax[1].set_xlim(-.2, 1)
plt.tight_layout()
glue('dist-req', fig, display=False)
plt.close()

In [34]:
fig, ax = plt.subplots(1, 2)
operations = {'échantillon':'nunique'}
feature_columns = ['fréquentation']

data = make_categorical_matrix(no_combined.copy(), feature_columns=feature_columns, operations=operations)
data2 = no_combined.groupby(['fréquentation', 'substrat'], as_index=False)['échantillon'].nunique()
ecdfd = no_combined.groupby(['échantillon', 'fréquentation', 'substrat'], as_index=False).pcs_m.sum()

sns.barplot(data=data, x='fréquentation', y='échantillon', color='grey', order=hue_order, ax=ax[0], label='fréquentation', alpha=0.4, zorder=1)
sns.barplot(data=data2, x='fréquentation', y='échantillon', hue='substrat',   edgecolor='white', linewidth=2, palette='tab20', ax=ax[0], order=hue_order, alpha=0.5, zorder=2)
caption = "L'utilisation de la plage et la granularité du substrat"
sns.ecdfplot(data=ecdfd, x='pcs_m', ax=ax[1], hue='fréquentation', palette='tab20', hue_order=hue_order)
ax[1].set_xlim(-.2, 1)
plt.tight_layout()
glue('freq-sub', fig, display=False)
plt.close()

In [35]:
fig, ax = plt.subplots(1, 2)

operations = {'échantillon':'nunique'}
feature_columns = ['fréquentation']

data = make_categorical_matrix(no_combined.copy(), feature_columns=feature_columns, operations=operations)
data2 = no_combined.groupby(['fréquentation', 'situation'], as_index=False)['échantillon'].nunique()
ecdfd = no_combined.groupby(['échantillon', 'fréquentation', 'situation'], as_index=False).pcs_m.sum()

sns.barplot(data=data, x='fréquentation', y='échantillon', color='grey', order=hue_order, ax=ax[0], label='fréquentation', alpha=0.4, zorder=1)
sns.barplot(data=data2, x='fréquentation', y='échantillon', hue='situation',   hue_order=["Campagne", "Urbain"], edgecolor='white', linewidth=2, palette='tab20', ax=ax[0], order=hue_order, alpha=0.5, zorder=2)

caption = "La distribution du nombre d'échantillons par situation et la fréquentation."
sns.ecdfplot(data=ecdfd, x='pcs_m', ax=ax[1], hue='fréquentation', palette='tab20', hue_order=hue_order)
ax[1].set_xlim(-.2, 1)
plt.tight_layout()
glue('freq-sit-corr', fig, display=False)
plt.close()

In [36]:
# !combined variables

data = make_categorical_matrix(data=f_comb.copy(), feature_columns=['fréquentation'])
hue_order = ['faible-moyenne', 'Elévée']
ecdfd = f_comb.groupby(['échantillon', 'fréquentation', 'substrat'], as_index=False).pcs_m.sum()

fig, ax = plt.subplots(1, 2)

sns.barplot(data=data, x='fréquentation', y='échantillon', color='grey', order=hue_order, ax=ax[0], label='fréquentation', alpha=0.4, zorder=1)
sns.barplot(data=f_freq_sub, x='fréquentation', y='échantillon', hue='substrat',   hue_order=["Sable", "Graviers"], edgecolor='white', linewidth=2, palette='tab20', ax=ax[0], order=hue_order, alpha=0.5, zorder=2)

caption = "Distribution du nombre d'échantillons par substrat et fréquentation, après fusion du substrat en deux groupes"

sns.ecdfplot(data=ecdfd, x='pcs_m', ax=ax[1], hue='fréquentation', palette='tab20', hue_order=hue_order)
ax[1].set_xlim(-.2, 1)
plt.tight_layout()
glue('freq-sub-cor', fig, display=False)
plt.close()

In [37]:
fig, ax = plt.subplots(1, 2)

data = make_categorical_matrix(data=f_comb.copy(), feature_columns=['fréquentation'])
ecdfd = f_comb.groupby(['échantillon', 'fréquentation', 'distance'], as_index=False).pcs_m.sum()
hue_order = ['faible-moyenne', 'Elévée']

sns.barplot(data=data, x='fréquentation', y='échantillon', color='grey', order=hue_order, ax=ax[0], label='fréquentation', alpha=0.4, zorder=1)
sns.barplot(data=f_freq_dist, x='fréquentation', y='échantillon', hue='distance',   hue_order=["<= 500 m", "> 500 m"], edgecolor='white', linewidth=2, palette='tab20', ax=ax[0], order=hue_order, alpha=0.5, zorder=2)

caption = "Distribution du nombre d'échantillons par distance au parking et fréquentation, après fusion de la distance en deux groupes."
sns.ecdfplot(data=ecdfd, x='pcs_m', ax=ax[1], hue='fréquentation', palette='tab20', hue_order=hue_order)
ax[1].set_xlim(-.2, 1)
plt.tight_layout()
glue('freq-dist-cor', fig, display=False)
plt.close()

(correlations)=
### Corrélation des conditions de prélèvement

::::{tab-set}

:::{tab-item} Corrélations
:selected:

Comme indiqué précédemment, les plages ayant un taux de fréquentation plus élevé ont également une valeur médiane plus élevée et sont principalement constituées de plages de sable.

Ces variables sont également corrélées. D'après le tableau ci-dessous, il y a une corrélation positive entre la fréquence et la situation. De plus, il y a une corrélation négative entre la distance et l'utilisation.


__Analyse des corrélations: Coefficient de Corrélation de Spearman :__

Il s'agit d'une mesure non paramétrique de la corrélation de rang. Il évalue dans quelle mesure la relation entre deux variables peut être décrite à l'aide d'une fonction monotone. Convient aux données ordinales car il dépend du rang des valeurs plutôt que de leurs magnitudes réelles.

{glue}`macro-correlation-one` 

__Les variables caractéristiques (conditions) sont ordinales.__ C'est-à-dire qu'elles sont classées de la plus faible à la plus forte. Par exemple, du moins fréquenté au plus fréquenté. _Pour le substrat, il s'agit d'une mesure de la taille de l'agrégat qui constitue la plage_.

Un substrat de 1 = sable fin, donc une corrélation négative entre le substrat et la fréquentation est une façon raffinée de dire que les gens aiment aller sur des plages sablonneuses. La corrélation négative entre la distance et la fréquentation signifie probablement que les gens préfèrent ne pas avoir à marcher trop loin.
::: 

:::{tab-item} Fréquentation et distance

{glue}`dist-req`

:::

:::{tab-item} Fréquentation et substrat

{glue}`freq-sub`
::: 

:::{tab-item} Fréquentation et situation

{glue}`freq-sit-corr`
:::

::::

#### Gérer les covariables indépendents

::::{tab-set}

:::{tab-item} Réduction de dimensions
:selected:


Notre intérêt est de prédire les valeurs sur les rives des lacs et de comprendre comment ou si l'une des variables indépendantes a un effet causal sur la variation de la densité des déchets.

La covariance des variables indépendantes peut avoir un effet de confusion sur l'interprétation des résultats et conduire à des prédictions inexactes. Certaines corrélations sont liées aux conditions d'échantillonnage, c'est-à-dire que les plages les plus fréquentées se trouvent dans des zones urbaines ou que les plages de sable sont les plus fréquentées. Il serait très difficile de classer ces corrélations comme causales. 

Pour réduire l'effet de ces corrélations non causatives, nous pouvons combiner certaines variables. Ici, nous avons combiné les variables du substrat et de la distance au parking, passant de quatre catégories à deux dans chaque cas.


__Réduction de dimensions en combinant des sous-groupes.__

1. Création d'une Variable Composite : combiner des covariables en une seule variable composite si elles représentent collectivement un phénomène sous-jacent unique. Cependant, cette approche simplifie le modèle et peut perdre certaines subtilités quant à la façon dont chaque variable affecte indépendamment la réponse.

:::

:::{tab-item} Composition des substrats

{glue}`freq-sub-cor`

Distribution du nombre d'échantillons par substrat et fréquentation, après fusion du substrat en deux groupes et fusion de fréquentation

:::

:::{tab-item} Composition des distances

{glue}`freq-dist-cor`

Distribution du nombre d'échantillons par distance au parking et fréquentation, après fusion de la distance en deux groupes et fusion de fréquentation.
:::

:::: 

In [38]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.utils import resample

from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder

from sklearn.model_selection import KFold

from sklearn.ensemble import GradientBoostingRegressor
from sklearn.svm import SVR

In [39]:
# from sklearn.model_selection import train_test_split
# from sklearn.ensemble import RandomForestRegressor
# from sklearn.utils import resample
# import numpy as np

def create_bins(predictions, bin_width=0.2):
    """
    Create bins from the predictions with a specified width.

    :param predictions: List or array of prediction values.
    :param bin_width: Width of each bin. Default is 0.2.
    :return: A tuple (bins, bin_counts).
        bins: The edges of the bins.
        bin_counts: The count of predictions in each bin.
    """
    # Determine the range for the bins
    max_prediction = max(predictions)
    bins = np.arange(0, max_prediction + bin_width, bin_width)

    # Count the number of predictions in each bin
    bin_counts, _ = np.histogram(predictions, bins=bins)

    return bins, bin_counts

def calculate_bin_probabilities(bin_counts):
    """
    Calculate the probability for each bin.

    :param bin_counts: The count of predictions in each bin.
    :return: List of probabilities for each bin.
    """
    total_predictions = sum(bin_counts)
    bin_probabilities = bin_counts / total_predictions
    return bin_probabilities


def analyze_scenario(scenario_data, func, n_iterations=100, bin_width=0.2):
    """
    Analyze a specific scenario using Random Forest regression with bootstrapping,
    and calculate feature importances.

    :param data: DataFrame containing the dataset.
    :param feature_1: The name of the first feature for filtering.
    :param feature_1_value: The value of the first feature to filter by.
    :param feature_2: The name of the second feature for filtering.
    :param feature_2_value: The value of the second feature to filter by.
    :param n_iterations: Number of bootstrap iterations. Default is 100.
    :param bin_width: Width of each bin for histogram. Default is 0.2.
    :return: A tuple containing bins, bin probabilities, flattened predictions, and feature importances.
    """
    
    # Prepare data for regression
    y_scaler = MinMaxScaler()
    y_scaled = y_scaler.fit_transform(scenario_data['pcs_m'].values.reshape(-1,1)).flatten()
    
    # Initialize the OneHotEncoder
    # here we encode the ordinal data
    encoder = OneHotEncoder(sparse_output=False)
    
    X = scenario_data.drop('pcs_m', axis=1)
    
    # Apply the encoder to the categorical columns
    encoded_data = encoder.fit_transform(scenario_data[['fréquentation', 'situation', 'distance', 'substrat']])
    # Create a DataFrame with the encoded data
    X_encoded = pd.DataFrame(encoded_data, columns=encoder.get_feature_names_out(['fréquentation', 'situation', 'distance', 'substrat']))

    
    X_train, X_test, y_train, y_test = train_test_split(X_encoded, y_scaled, test_size=0.2, random_state=42)

    # Bootstrap predictions and accumulate feature importances
    bootstrap_predictions = []
    feature_importances_accumulated = np.zeros(X_train.shape[1])
    
    # Collect diagnostic at each repetition
    cum_mse = []
    cum_r2 = []
    
    for _ in range(n_iterations):
        X_train_sample, y_train_sample = resample(X_train, y_train)
        rf_model_sample = func
        rf_model_sample.fit(X_train_sample, y_train_sample)
        
        # the results of this prediction are tested against the original
        # y_test
        pred = rf_model_sample.predict(X_test)
        
        r2 = r2_score(y_test, pred)
        pred = y_scaler.inverse_transform(pred.reshape(-1, 1)).flatten()
        bootstrap_predictions.append(pred)
        mse = mean_squared_error(y_test , pred)
        
        
        feature_importances_accumulated += rf_model_sample.feature_importances_
        
        cum_mse.append(mse)
        cum_r2.append(r2)
    
    cv_mse = []
    
    
    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    
    for train_index, val_index in kf.split(X_encoded):
        X_train_kfold, X_val_kfold = X_encoded.iloc[train_index], X_encoded.iloc[val_index]
        y_train_kfold, y_val_kfold = y_scaled[train_index], y_scaled[val_index]

        model_kfold = func
        model_kfold.fit(X_train_kfold, y_train_kfold)
        y_pred_kfold = model_kfold.predict(X_val_kfold)
        
        y_oscale_t = y_scaler.inverse_transform(y_val_kfold.reshape(-1, 1)).flatten()
        y_oscale_p = y_scaler.inverse_transform(y_pred_kfold.reshape(-1,1)).flatten()

        mse = mean_squared_error(y_oscale_t, y_oscale_p)
        r2 = r2_score(y_oscale_t, y_oscale_p)

        cv_mse.append(mse)
        


    # Average feature importances
    feature_importances = feature_importances_accumulated / n_iterations

    # Flatten the predictions array
    predictions_flat = np.array(bootstrap_predictions).flatten()

    # Create bins and calculate bin probabilities
    bins, bin_counts = create_bins(predictions_flat, bin_width)
    bin_probabilities = calculate_bin_probabilities(bin_counts)

    return predictions_flat, feature_importances, cum_mse, cum_r2, cv_mse, bins, bin_probabilities

def plot_histogram(predictions, observed, title="", reference='camp-dist-1', display=False):
    fig, ax = plt.subplots(figsize=(10, 6))
    sns.histplot(predictions, bins=20, stat="probability", ax=ax, label='prédictions', zorder=1)
    sns.histplot(observed, bins=20, stat="probability", label='observée', zorder=0, ax=ax)
    plt.title(title, loc='left')
    plt.xlabel('pcs/m')
    plt.ylabel('Densité de Probabilité')
    plt.legend()
    glue(reference, fig, display=display)
    plt.close()
    
def most_common_for_scenario(data, caption, boundaries: dict = boundaries, top_label: list = top_label, language: str = 'fr', lang_maps: dict = language_maps):
    camp_close = rc.ReportClass(data, boundaries=boundaries, language=language, lang_maps=lang_maps, top_label=top_label)
    c_mcommon, weights = camp_close.most_common
    x1 = rc.translated_and_style_for_display(c_mcommon, lang_maps[language], language, gradient=False).set_caption(caption)
    return x1

def evalutate_model(r2s, mses, label, model='random-forest'):
    r2 = np.round(np.mean(r2s), 2)
    mse = np.round(np.mean(mses), 2)
    results = {"cross validated error":r2, "mean² error":mse, 'model':model}
    return pd.DataFrame(results, index=[label])

# Calculating quantiles for Scenario 2
q_uants = [0.01, 0.25, 0.5, 0.75, 0.99]
index = ['1%', '25%', '50%', '75%', '99%', 'Moyenne']
def makeqdf(observed, predicted, index=index, quants=q_uants, caption=""):
    
    o_q = np.quantile(observed, quants)
    m_o = np.mean(observed)
    o_p = np.quantile(predicted, quants)
    m_p = np.mean(predicted)
    
    results = {'observée':[*o_q, m_o], 'prédiction': [*o_p, m_p]}
    return pd.DataFrame(results, index=index).style.set_table_styles(table_css_styles_top).format(precision=2).set_caption(caption)

cols = ['échantillon', 'fréquentation','situation', 'distance', 'substrat']
# data=pd.read_csv('plastock_with _asl_landuse.csv')
# data=data.groupby(cols, as_index=False).pcs_m.sum()
# scenario_2_data = data[(data['situation'] == 2) & (data['fréquentation'] == 3)].copy()

In [40]:
# Filter for Scenario 
test_xi = f_combi[(f_combi['situation'] == 2) & (f_combi['fréquentation'] == 3)].copy()
test_x = test_xi.groupby(cols, as_index=False).pcs_m.sum()
test_x = test_x[['fréquentation', 'situation', 'distance', 'substrat', 'pcs_m']]

# model parameters
estimators = 10
iterations = 500

func = RandomForestRegressor(n_estimators=estimators, criterion="absolute_error", random_state=42)
predictions, feature_importance, mse, r2, cv_mse, bins, bin_probs = analyze_scenario(test_x, func,  n_iterations=iterations, bin_width=0.2)

caption = 'Urban, Fréquentation Elévée'
q_sit_2_freq_3 = makeqdf(test_x.pcs_m.values, predictions, caption=caption)
glue('q-hf-ville', q_sit_2_freq_3, display=False)

# the most common objects for this scenario
data = test_xi.rename(columns={'échantillon':'loc_date'})
caption = "Les objets les plus courants, situation = ville, fréquentation = élévée"
mcx = most_common_for_scenario(data, caption)
glue('ville-hf-mc', mcx, display=False) 

# the histogram for this scenario:
title = 'Plastock 2022, Le Léman\nDistribution des Prédictions - Situation Ville, Haute Fréquentation'
plot_histogram(predictions, test_x.pcs_m.values, title=title, reference='ville-hf', display=False)

evals = []

model_evaluation = evalutate_model(cv_mse, mse, "Ville et haute fréquentation")
evals.append(model_evaluation)

In [41]:
# Filter for Scenario 
test_xi = f_combi[(f_combi['situation'] == 1) & (f_combi['fréquentation'] == 3)].copy()
test_x = test_xi.groupby(cols, as_index=False).pcs_m.sum()
test_x = test_x[['fréquentation', 'situation', 'distance', 'substrat', 'pcs_m']]

func = RandomForestRegressor(n_estimators=estimators, criterion="absolute_error", random_state=42)
predictions, feature_importance, mse, r2, cv_mse, bins, bin_probs = analyze_scenario(test_x, func,  n_iterations=iterations, bin_width=0.2)

# the quantiles for this scenario
caption="Campagne, Fréquentation Eléveé"
q_sit_1_freq_3 = makeqdf(test_x.pcs_m.values, predictions, caption=caption)
glue('q-hf-camp', q_sit_1_freq_3, display=False)

# the most common objects for this scenario
data = test_xi.rename(columns={'échantillon':'loc_date'})
mcx = most_common_for_scenario(data, caption)
glue('camp-hf-mc', mcx, display=False) 

# the histogram for this scenario:
title = 'Plastock 2022, Le Léman\nDistribution des Prédictions - Situation Campagne, Haute Fréquentation\n'
plot_histogram(predictions, test_x.pcs_m.values, title=title, reference='camp-hf', display=False)

model_evaluation2 = evalutate_model(cv_mse, mse, "Campagne et haut fréquentation")
evals.append(model_evaluation2)

In [42]:
# Filter for Scenario 
test_xi = f_combi[(f_combi['situation'] == 1) & (f_combi['distance'] == 1)].copy()
test_x = test_xi.groupby(cols, as_index=False).pcs_m.sum()
test_x = test_x[['fréquentation', 'situation', 'distance', 'substrat', 'pcs_m']]


func = RandomForestRegressor(n_estimators=estimators, criterion="absolute_error", random_state=42)
predictions, feature_importance, mse, r2, cv_mse, bins, bin_probs = analyze_scenario(test_x, func,  n_iterations=iterations, bin_width=0.2)

# the quantiles for this scenario
caption = 'Campagne, <= 500 m du parking'
q_sit_1_d_1 = makeqdf(test_x.pcs_m.values, predictions, caption=caption)
glue('q-camp-dist_1', q_sit_1_d_1, display=False)

# the most common objects for this scenario
data = test_xi.rename(columns={'échantillon':'loc_date'})
mcx = most_common_for_scenario(data, caption)
glue('camp-dist1-mc', mcx, display=False) 

# the histogram for this scenario:
title = 'Plastock 2022, Le Léman\nDistribution des Prédictions - Situation Campagne, distance < 500 m'
plot_histogram(predictions, test_x.pcs_m.values, title=title, reference='camp-dist-1', display=False)

model_evaluation3 = evalutate_model(cv_mse, mse, "Campagne et dist < 500 m")
evals.append(model_evaluation3)

In [43]:
# Filter for Scenario 
test_xi = f_combi[(f_combi['situation'] == 2) & (f_combi['distance'] == 1)].copy()
test_x = test_xi.groupby(cols, as_index=False).pcs_m.sum()
test_x = test_x[['fréquentation', 'situation', 'distance', 'substrat', 'pcs_m']]

func = RandomForestRegressor(n_estimators=estimators, criterion="absolute_error", random_state=42)
predictions, feature_importance, mse, r2, cv_mse, bins, bin_probs = analyze_scenario(test_x, func,  n_iterations=iterations, bin_width=0.2)

# the quantiles for this scenario
caption = 'Urban, <= 500 m du parking'
q_sit_2_d_1 = makeqdf(test_x.pcs_m.values, predictions, caption=caption)
glue('q-ville-dist_1', q_sit_2_d_1, display=False)

# the most common objects for this scenario
data = test_xi.rename(columns={'échantillon':'loc_date'})
mcx = most_common_for_scenario(data, caption)
glue('ville-dist1-mc', mcx, display=False) 

# the histogram for this scenario:
title = 'Plastock 2022, Le Léman\nDistribution des Prédictions - Situation Ville, distance < 500 m'
plot_histogram(predictions, test_x.pcs_m.values, title=title, reference='ville-dist-1', display=False)

model_evaluation4 = evalutate_model(cv_mse, mse, "Ville et dist < 500 m")
evals.append(model_evaluation4)

In [44]:
# Filter for Scenario 
test_xi = f_combi[(f_combi['substrat'] == 1)].copy()
test_x = test_xi.groupby(cols, as_index=False).pcs_m.sum()
test_x = test_x[['fréquentation', 'situation', 'distance', 'substrat', 'pcs_m']]
func = RandomForestRegressor(n_estimators=estimators, criterion="absolute_error", random_state=42)
predictions, feature_importance, mse, r2, cv_mse, bins, bin_probs = analyze_scenario(test_x, func,  n_iterations=iterations, bin_width=0.2)

# the quantiles for this scenario
caption = 'Sables'
q_sub_1 = makeqdf(test_x.pcs_m.values, predictions, caption=caption)
glue('q_subs_1', q_sub_1, display=False)

# the most common objects for this scenario
data = test_xi.rename(columns={'échantillon':'loc_date'})
mcx = most_common_for_scenario(data, caption)
glue('subs_1_mc', mcx, display=False) 

# the histogram for this scenario:
title = 'Plastock 2022, Le Léman\nDistribution des Prédictions - Sables'
plot_histogram(predictions, test_x.pcs_m.values, title=title, reference='subs_1_hist', display=False)

model_evaluation5 = evalutate_model(cv_mse, mse, "Gravier")
evals.append(model_evaluation5)

In [45]:
# Filter for Scenario 
test_xi = f_combi[(f_combi['substrat'] == 2)].copy()
test_x = test_xi.groupby(cols, as_index=False).pcs_m.sum()
test_x = test_x[['fréquentation', 'situation', 'distance', 'substrat', 'pcs_m']]
func = RandomForestRegressor(n_estimators=estimators, criterion="absolute_error", random_state=42)
predictions, feature_importance, mse, r2, cv_mse, bins, bin_probs = analyze_scenario(test_x, func,  n_iterations=iterations, bin_width=0.2)

# the quantiles for this scenario
caption='Graviers'
q_sub_2 = makeqdf(test_x.pcs_m.values, predictions, caption=caption)
glue('q_subs_2', q_sub_2, display=False)

# the most common objects for this scenario
data = test_xi.rename(columns={'échantillon':'loc_date'})
mcx = most_common_for_scenario(data, caption)
glue('subs_2_mc', mcx, display=False) 

# the histogram for this scenario:
title = 'Plastock 2022, Le Léman\nDistribution des Prédictions - Graviers'
plot_histogram(predictions, test_x.pcs_m.values, title=title, reference='subs_2_hist', display=False)

model_evaluation6 = evalutate_model(cv_mse, mse, "Sable")
evals.append(model_evaluation6)

In [46]:
# Filter for Scenario 
# This is all the values => no filter
# just aggregating to the sample_id 
test_xi = f_combi.copy()
test_x = test_xi.groupby(cols, as_index=False).pcs_m.sum()

func = RandomForestRegressor(n_estimators=estimators, criterion="absolute_error", random_state=42)
predictions, feature_importance, mse, r2, cv_mse, bins, bin_probs = analyze_scenario(test_x, func,  n_iterations=iterations, bin_width=0.2)

# the quantiles for this scenario
caption = 'Toutes les conditions'
q_tous = makeqdf(test_x.pcs_m.values, predictions, caption=caption)
glue('q-tous', q_tous, display=False)

# the most common objects for this scenario
data = test_xi.rename(columns={'échantillon':'loc_date'})
mcx = most_common_for_scenario(data, caption)
glue('tous-mc', mcx, display=False) 

# the histogram for this scenario:
title = 'Plastock 2022, Le Léman\nDistribution des Prédictions'
plot_histogram(predictions, test_x.pcs_m.values, title=title, reference='tous', display=False)

model_evaluationx = evalutate_model(r2, mse, "Tous")
evals.append(model_evaluationx)

ev_df = pd.concat(evals[::-1])
glue('model-evals', ev_df.style.set_table_styles(table_css_styles).format(precision=2), display=False)

(random_forest)=
### Régression avec Forêt Aléatoire (Random Forest Regression)

Source : [scikit-learn random forest](https://scikit-learn.org/0.16/modules/generated/sklearn.ensemble.RandomForestRegressor.html)

criterion : `absolute error`

La régression avec forêt aléatoire est une technique d'apprentissage automatique (machine learning) utilisée pour prédire des résultats continus (par opposition aux catégories dans la classification). C'est une méthode d'apprentissage ensembliste, ce qui signifie qu'elle combine les prédictions de plusieurs algorithmes d'apprentissage automatique pour produire des prédictions plus précises.

::::{tab-set}

:::{tab-item} Toutes les conditions
{glue}`tous`

|Plus courants|Quantiles|
|:-:|:-:|
|{glue}`tous-mc`|{glue}`q-tous`|
:::

:::{tab-item} Graviers
{glue}`subs_2_hist`

|Plus courants|Quantiles|
|:-:|:-:|
|{glue}`subs_2_mc`|{glue}`q_subs_2`|
:::

:::{tab-item} Sables
{glue}`subs_1_hist`

|Plus courants|Quantiles|
|:-:|:-:|
|{glue}`subs_1_mc`|{glue}`q_subs_1`|
:::

:::{tab-item} Ville et haute Fréquentation
{glue}`ville-hf`

|Plus courants|Quantiles|
|:-:|:-:|
|{glue}`ville-hf-mc`|{glue}`q-hf-ville`|
:::

:::{tab-item} Campagne et haute fréquentation
{glue}`camp-hf`

|Plus courants|Quantiles|
|:-:|:-:|
|{glue}`camp-hf-mc`|{glue}`q-hf-camp`|
:::

:::{tab-item} Campagne et parking <= 500 m
{glue}`camp-dist-1`

|Plus courants|Quantiles|
|:-:|:-:|
|{glue}`camp-dist1-mc`|{glue}`q-camp-dist_1`|
:::

:::{tab-item} Ville et parking <= 500 m
{glue}`ville-dist-1`

|Plus courants|Quantiles|
|:-:|:-:|
|{glue}`ville-dist1-mc`|{glue}`q-ville-dist_1`|
:::



:::{tab-item} Résultats
:selected:

````{grid} 1 2 2 2

```{grid-item}
{glue}`q-tous`
```

```{grid-item}

Les modèles ont fait l'objet d'un bootstrap, 100 itérations pour chaque scénario. Les résultats estimés sont la collection de toutes les prédictions de chaque itération.

Par exemple, le tableau intitulé "Gravier" présente les résultats observés et prévus pour les plages ayant un substrat de 3 ou 4.

```

```{grid-item}
{glue}`q_subs_2`
```

```{grid-item}
{glue}`q_subs_1`
```

```{grid-item}
{glue}`q-hf-ville`
```

```{grid-item}
{glue}`q-hf-camp`
```

```{grid-item}
{glue}`q-camp-dist_1`
```

```{grid-item}
{glue}`q-ville-dist_1`
```

````
:::

::::
    




<!-- 
::::{tab-set}

:::{tab-item} Modèles
:selected:

La régression avec forêt aléatoire est une technique d'apprentissage automatique (machine learning) utilisée pour prédire des résultats continus (par opposition aux catégories dans la classification). C'est une méthode d'apprentissage ensembliste, ce qui signifie qu'elle combine les prédictions de plusieurs algorithmes d'apprentissage automatique pour produire des prédictions plus précises que n'importe quel modèle individuel.

Basée sur des __Arbres de Décision__ : Une forêt aléatoire est composée de nombreux arbres de décision, qui sont des modèles simples faisant des prédictions basées sur une série de choix binaires. Chaque arbre dans une forêt aléatoire donne une prédiction, et la forêt aléatoire combine ces prédictions pour produire un résultat plus précis.

|Tous|Sable|
|:-:|:-:|
|{glue}`q-tous`|{glue}`q_subs_2`

:::

:::{tab-item} Tous les variables
{glue}`tous`

|Plus courants|Quantiles|
|:-:|:-:|
|{glue}`tous-mc`|{glue}`q-tous`|
:::

:::{tab-item} Sabloneuse
{glue}`subs_2_hist`

|Plus courants|Quantiles|
|:-:|:-:|
|{glue}`subs_2_mc`|{glue}`q_subs_2`|
:::

:::{tab-item} Graviers
{glue}`subs_1_hist`

|Plus courants|Quantiles|
|:-:|:-:|
|{glue}`subs_1_mc`|{glue}`q_subs_1`|
:::

:::{tab-item} Ville et haute Fréquentation
{glue}`ville-hf`

|Plus courants|Quantiles|
|:-:|:-:|
|{glue}`ville-hf-mc`|{glue}`q-hf-ville`|
:::

:::{tab-item} Campagne et haute fréquentation
{glue}`camp-hf`

|Plus courants|Quantiles|
|:-:|:-:|
|{glue}`camp-hf-mc`|{glue}`q-hf-camp`|
:::

:::{tab-item} Campagne et parking < 500 m
{glue}`camp-dist-1`

|Plus courants|Quantiles|
|:-:|:-:|
|{glue}`camp-dist1-mc`|{glue}`q-camp-dist_1`|
:::

:::{tab-item} Ville et parking < 500 m
{glue}`ville-dist-1`

|Plus courants|Quantiles|
|:-:|:-:|
|{glue}`ville-dist1-mc`|{glue}`q-ville-dist_1`|
:::
:::: -->

In [47]:
from typing import Type, Optional, Callable
from typing import List, Dict, Union, Tuple

def sum_a_b(zipped):
    for element in zipped:
        # the new beta distribution would be
        # total success, (total tries - total success)
        new_element_0 = np.array([np.array([x[0], x[1] - x[0]]) for x in element[0]])
        new_element_1 = np.array([x for x in element[1]])
        t3 = new_element_0 + new_element_1
        
        yield t3

# Grid approximation

grid_val_index = np.linspace(0, 19.99, 2000)
groupby_columns = ['sample_id', 'location', 'date', 'city', 'orchards', 'vineyards', 'buildings', 'forest',
                   'undefined', 'public_services', 'streets']
def draw_a_beta_value(generator):
    d = next(generator)
    # drawing a random number from the beta distribution
    # this is the the chance p, that a binomial distribution will
    # result in True.
    my_beta = [beta(x[0], x[1]).rvs(1) for x in d]
    yield my_beta


def binomial_probability_of_failure(generator):
    # in this case failure means exceeding the value
    # for trash a success is never exceeding the value
    d = next(generator)
    di = [x[0] for x in d]
    yield di

def bin_land_use_values(*, data: pd.DataFrame, column: str, num_bins: int = 4) -> pd.DataFrame:
    """
    Bins the specified column's values into a given number of bins and adds a new column to the DataFrame with these bin labels.

    Args:
        data (pd.DataFrame): The DataFrame to modify.
        column (str): The name of the column to bin.
        num_bins (int, optional): The number of bins to use. Defaults to 20.

    Returns:
        pd.DataFrame: The modified DataFrame with an additional column for binned values.
    """
    data[f'{column}_bin'] = pd.cut(data[column], bins=num_bins, labels=[1, 2, 3, 4 ], include_lowest=True)
    return data


def calculate_likelihood(*, aggregated_data: pd.DataFrame, bin_density_column: str, pcs_column: str = 'pcs/m',
                         grid_range: np.ndarray = None, bins: list = None) -> pd.DataFrame:
    """
    Calculates the likelihood of observing the aggregated pcs/m data for each grid point and bin density value.

    Args:
        aggregated_data (pd.DataFrame): The aggregated data to be used for likelihood calculation.
        bin_density_column (str): The column representing bin density numbers.
        pcs_column (str, optional): The pcs/m column to use for calculation. Defaults to 'pcs/m'.
        grid_range (np.ndarray, optional): The range of grid values. Defaults to np.linspace(0, 9.99, 1000).

    Returns:
        pd.DataFrame: A DataFrame with likelihood values for each grid value and bin density number.
    """
    likelihood_df = pd.DataFrame(index=grid_range)
    
    for bin_value in bins:
        bin_data = aggregated_data[aggregated_data[bin_density_column] == bin_value]
        if bin_data.empty:
            likelihoods = [np.array([1, 1]) for grid_point in grid_range]
        else:
            likelihoods = [np.array([(bin_data[pcs_column] > grid_point).sum(), len(bin_data)]) for grid_point in
                           grid_range]
        likelihood_df[f'Likelihood_{bin_value}'] = likelihoods
    return likelihood_df

def calculate_beta_prior(*, grid_range: np.ndarray = grid_val_index, bin_density_numbers: List[int] = list(range(1,
                                                                                                    21))) -> pd.DataFrame:
    """
    Calculates a Beta(1, 1) prior for each value in the specified grid range for each bin density number.

    Args:
        grid_range (np.ndarray, optional): The range of grid values. Defaults to np.linspace(0, 9.99, 1000).
        bin_density_numbers (List[int], optional): List of bin density numbers. Defaults to range(1, 21).

    Returns:
        pd.DataFrame: A DataFrame with Beta(1, 1) prior values for each grid value and bin density number.
    """
    prior_df = pd.DataFrame(index=grid_range)
    prior_values = np.array([1, 1])  # Constant value since Beta(1, 1) is uniform
    
    for bin_number in bin_density_numbers:
        prior_df[f'Bin_{bin_number}'] = [prior_values for grid_point in grid_range]
    return prior_df

class GridApproximation:
    posterior = []
    groupby_columns = groupby_columns
    
    def __init__(self, data: pd.DataFrame, these_codes: list[str] = None, value_column: str = 'pcs/m',
                 land_use_column: str = 'buildings', n_bins: int = 5, groupby_columns: list = groupby_columns):
        self.data = data
        self.these_codes = these_codes
        self.value_column = value_column
        self.land_use_column = land_use_column
        self.n_bins = n_bins
        self.groupby_columns = groupby_columns
    
    @property
    def sample_totals(self):
        aare_dt = self.data[self.data.code.isin(self.these_codes)].groupby(self.groupby_columns, as_index=False)[
            self.value_column].sum()
        return aare_dt
    
    @property
    def binned_samples(self):
        new_bins = bin_land_use_values(data=self.sample_totals, column=self.land_use_column, num_bins=self.n_bins)
        return new_bins

    @property
    def prior_grid(self, afunc: Callable = calculate_beta_prior):
        prior_grid = afunc(bin_density_numbers=list(range(1, self.n_bins + 1)))
        return prior_grid

    @property
    def likelihood_grid(self):
        l_grid = calculate_likelihood(aggregated_data=self.binned_samples,
                                            bin_density_column=f'{self.land_use_column}_bin',
                                            pcs_column=self.value_column)
        return l_grid

    @property
    def posterior_grid(self):
        ti = np.array(self.likelihood_grid.values)
        t2 = np.array(self.prior_grid.values)
        grid_val_index = self.prior_grid.index
    
        zd = list(zip(ti, t2))
        t = sum_a_b(zd)
    
        posteriors = []
    
        for i in grid_val_index:
            st = binomial_probability_of_failure(draw_a_beta_value(t))
            val = next(st)
            posteriors.append(val)
    
        pts = pd.DataFrame(posteriors, index=grid_val_index, columns=self.prior_grid.columns)
        pts['pcs'] = pts.index
        plg = pd.melt(pts, id_vars='pcs', value_vars=pts.columns)
        return plg

In [48]:
from scipy.stats import beta
from scipy.stats import multinomial

def define_posterior(likelihood, prior, grid_val_index: np.array = None):
    
    # the alpha, beta parameters of the likelihood and prior are assembled
    alpha_beta = list(zip(likelihood.values, prior.values))
    a_b_sum = sum_a_b(alpha_beta)
    
    posteriors = []
    for i in grid_val_index:
        # the sum of successes and failures for the scenario at the given
        # grid value are used as the alpha, beta parameters of the beta distribtion
        # for the binomial/bernouli probability that a sample will exceed the grid
        # value i.
        st = binomial_probability_of_failure(draw_a_beta_value(a_b_sum))
        val = next(st)
        posteriors.append(val)
    
    # return posterior probabilities with gird index and column labels
    post_grid_pstock = pd.DataFrame(posteriors, index=grid_val_index, columns=prior.columns)
    
    # identify the x scale of the grid
    post_grid_pstock['X'] = post_grid_pstock.index
    
    # this column is the normalized probabilities that a sample
    # will exceed a value on the grid.
    post_grid_pstock['norm'] = post_grid_pstock['Bin_1']/post_grid_pstock['Bin_1'].sum()
    
    return post_grid_pstock

def non_zero(alist):
    # find the first non-zero object in an array
    # return the index number and the value.
    for i, anum in enumerate(alist):
        if anum != 0:
            return i, anum
    return None

def draw_sample_from_multinomial(normed, n=100):
    # the norm column from the posterior data frame is
    # used as the probabilities of a multinomial distribution
    rv = multinomial(1, normed.values)
    y = rv.rvs(n)   

    indexes = []
    for i in range(0, len(y)):
        indexes.append(non_zero(y[i])[0])
    return indexes


def posterior_predictions(p_g_p):
    
    p_norm = p_g_p['norm']
    
    indexes = draw_sample_from_multinomial(p_norm)
    results_scale = p_g_p.reset_index(drop=True)
    sample_totals = results_scale.loc[indexes, "X"]
    
    return sample_totals

# the prior data from surveys
iqaasl_prior = report_iq_pk.w_df[report_iq_pk.w_df.project == "IQAASL"].copy()
iq_p = iqaasl_prior.groupby(['loc_date', 'project'], as_index=False).pcs_m.sum()
iq_p['top'] = 1

iq_prior = calculate_likelihood(aggregated_data=iq_p, bin_density_column='top', pcs_column='pcs_m', grid_range=grid_val_index, bins=[1])
iq_prior.rename(columns={'Likelihood_1':'Bin_1'}, inplace=True)

# assuming know prior knowledge
beta_prior = calculate_beta_prior(bin_density_numbers=[1])
    
col = 'top'
pcs_col = 'pcs_m'
grid_range = grid_val_index
bins = [1]

test_x = f_combi.copy().groupby(cols, as_index=False).pcs_m.sum()
test_x['top'] = 1

grid_pstock = calculate_likelihood(aggregated_data=test_x, bin_density_column=col, pcs_column=pcs_col, grid_range=grid_range, bins=bins)

# posterior uninformed
post_grid_pstock = define_posterior(grid_pstock, beta_prior, grid_val_index=grid_range)

# posterior informed
post_grid_iqp =define_posterior(grid_pstock, iq_prior, grid_val_index=grid_range)

# samples
sample_totals = posterior_predictions(post_grid_pstock.copy())
s_iqp = posterior_predictions(post_grid_iqp.copy())

caption = 'Toutes les conditions'

test_grid_quants = makeqdf(test_x.pcs_m.values, s_iqp, caption=caption)
glue('q-tous-b', test_grid_quants, display=False)
title = 'Plastock 2022, Le Léman\nDistribution des Prédictions: toutes les conditions,  grid approximation, prior = IQAASL'

plot_histogram(s_iqp, test_x.pcs_m.values, title=title, reference='toutes-gapprox', display=False)



In [49]:
col = 'substrat'
pcs_col = 'pcs_m'
grid_range = grid_val_index
bins = [2]

test_xi = f_combi[(f_combi['substrat'] == 2)].copy()
test_x = test_xi.groupby(cols, as_index=False).pcs_m.sum()


grid_pstock = calculate_likelihood(aggregated_data=test_x, bin_density_column=col, pcs_column=pcs_col, grid_range=grid_range, bins=bins)

# posterior uninformed
post_grid_pstock = define_posterior(grid_pstock, beta_prior, grid_val_index=grid_range)

# posterior informed
post_grid_iqp =define_posterior(grid_pstock, iq_prior, grid_val_index=grid_range)

# samples
sample_totals = posterior_predictions(post_grid_pstock.copy())
s_iqp = posterior_predictions(post_grid_iqp.copy())

caption = 'Graviers'

test_grid_quants = makeqdf(test_x.pcs_m.values, sample_totals, caption=caption)
glue('q-gravier-b', test_grid_quants, display=False)
title = 'Plastock 2022, Le Léman\nDistribution des Prédictions: Graviers, grid approximation, prior = IQAASL'

plot_histogram(sample_totals, test_x.pcs_m.values, title=title, reference='graviers-gapprox', display=False)

In [50]:
col = 'substrat'
pcs_col = 'pcs_m'
grid_range = grid_val_index
bins = [1]

test_xi = f_combi[(f_combi['substrat'] == 1)].copy()
test_x = test_xi.groupby(cols, as_index=False).pcs_m.sum()


grid_pstock = calculate_likelihood(aggregated_data=test_x, bin_density_column=col, pcs_column=pcs_col, grid_range=grid_range, bins=bins)

# posterior uninformed
post_grid_pstock = define_posterior(grid_pstock, beta_prior, grid_val_index=grid_range)

# posterior informed
post_grid_iqp =define_posterior(grid_pstock, iq_prior, grid_val_index=grid_range)

# samples
sample_totals = posterior_predictions(post_grid_pstock.copy())
s_iqp = posterior_predictions(post_grid_iqp.copy())

caption = 'Sable'

test_grid_quants = makeqdf(test_x.pcs_m.values, sample_totals, caption=caption)
glue('q-sable-b', test_grid_quants, display=False)
title = 'Plastock 2022, Le Léman\nDistribution des Prédictions: Sable, grid approximation, prior = IQAASL'

plot_histogram(sample_totals, test_x.pcs_m.values, title=title, reference='sables-gapprox', display=False)

In [51]:
col = 'fréquentation'
pcs_col = 'pcs_m'
grid_range = grid_val_index
bins = [3]

test_xi = f_combi[(f_combi['situation'] == 2) & (f_combi['fréquentation'] == 3)].copy()
test_x = test_xi.groupby(cols, as_index=False).pcs_m.sum()


grid_pstock = calculate_likelihood(aggregated_data=test_x, bin_density_column=col, pcs_column=pcs_col, grid_range=grid_range, bins=bins)

# posterior uninformed
post_grid_pstock = define_posterior(grid_pstock, beta_prior, grid_val_index=grid_range)

# posterior informed
post_grid_iqp =define_posterior(grid_pstock, iq_prior, grid_val_index=grid_range)

# samples
sample_totals = posterior_predictions(post_grid_pstock.copy())
s_iqp = posterior_predictions(post_grid_iqp.copy())

caption = 'Ville et haut fréquentation'

test_grid_quants = makeqdf(test_x.pcs_m.values, sample_totals, caption=caption)
glue('q-v-hf-b', test_grid_quants, display=False)
title = 'Plastock 2022, Le Léman\nDistribution des Prédictions: Ville et haut fréquentation, grid approximation'

plot_histogram(sample_totals, test_x.pcs_m.values, title=title, reference='v-hf-gapprox', display=False)

In [52]:
col = 'fréquentation'
pcs_col = 'pcs_m'
grid_range = grid_val_index
bins = [3]

test_xi = f_combi[(f_combi['situation'] == 1) & (f_combi['fréquentation'] == 3)].copy()
test_x = test_xi.groupby(cols, as_index=False).pcs_m.sum()


grid_pstock = calculate_likelihood(aggregated_data=test_x, bin_density_column=col, pcs_column=pcs_col, grid_range=grid_range, bins=bins)

# posterior uninformed
post_grid_pstock = define_posterior(grid_pstock, beta_prior, grid_val_index=grid_range)

# posterior informed
post_grid_iqp =define_posterior(grid_pstock, iq_prior, grid_val_index=grid_range)

# samples
sample_totals = posterior_predictions(post_grid_pstock.copy())
s_iqp = posterior_predictions(post_grid_iqp.copy())

caption = 'Campagne et haut fréquentation'

test_grid_quants = makeqdf(test_x.pcs_m.values, sample_totals, caption=caption)
glue('q-cam-hf-b', test_grid_quants, display=False)
title = 'Plastock 2022, Le Léman\nDistribution des Prédictions: Campagne et haut fréquentation, grid approximation'

plot_histogram(sample_totals, test_x.pcs_m.values, title=title, reference='cam-hf-gapprox', display=False)

In [53]:
col = 'situation'
pcs_col = 'pcs_m'
grid_range = grid_val_index
bins = [2]

test_xi = f_combi[(f_combi['situation'] == 2) & (f_combi['distance'] == 1)].copy()
test_x = test_xi.groupby(cols, as_index=False).pcs_m.sum()


grid_pstock = calculate_likelihood(aggregated_data=test_x, bin_density_column=col, pcs_column=pcs_col, grid_range=grid_range, bins=bins)

# posterior uninformed
post_grid_pstock = define_posterior(grid_pstock, beta_prior, grid_val_index=grid_range)

# posterior informed
post_grid_iqp =define_posterior(grid_pstock, iq_prior, grid_val_index=grid_range)

# samples
sample_totals = posterior_predictions(post_grid_pstock.copy())
s_iqp = posterior_predictions(post_grid_iqp.copy())

caption = 'Ville et distance <= 500 m'

test_grid_quants = makeqdf(test_x.pcs_m.values, sample_totals, caption=caption)
glue('q-v-dist1-b', test_grid_quants, display=False)
title = 'Plastock 2022, Le Léman\nDistribution des Prédictions: Ville et distance <= 500 m, grid approximation, prior = IQAASL'

plot_histogram(sample_totals, test_x.pcs_m.values, title=title, reference='v-dist1-gapprox', display=False)

In [54]:
col = 'situation'
pcs_col = 'pcs_m'
grid_range = grid_val_index
bins = [1]

test_xi = f_combi[(f_combi['situation'] == 1) & (f_combi['distance'] == 1)].copy()
test_x = test_xi.groupby(cols, as_index=False).pcs_m.sum()


grid_pstock = calculate_likelihood(aggregated_data=test_x, bin_density_column=col, pcs_column=pcs_col, grid_range=grid_range, bins=bins)

# posterior uninformed
post_grid_pstock = define_posterior(grid_pstock, beta_prior, grid_val_index=grid_range)

# posterior informed
post_grid_iqp =define_posterior(grid_pstock, iq_prior, grid_val_index=grid_range)

# samples
sample_totals = posterior_predictions(post_grid_pstock.copy())
s_iqp = posterior_predictions(post_grid_iqp.copy())

caption = 'Campagne et distance <= 500 m'

test_grid_quants = makeqdf(test_x.pcs_m.values, sample_totals, caption=caption)
glue('q-c-dist1-b', test_grid_quants, display=False)
title = 'Plastock 2022, Le Léman\nDistribution des Prédictions: Campagne et distance <= 500 m, grid approximation, prior = IQAASL'

plot_histogram(sample_totals, test_x.pcs_m.values, title=title, reference='c-dist1-gapprox', display=False)

In [55]:
# fig, ax = plt.subplots()

# sns.histplot(sample_totals.values, ax=ax, label='prédictions')
# sns.histplot(test_x.pcs_m.values, ax=ax, label='observée')
# ax.legend()

__Approximation Bayésienne par Grille__



(grid_approx)=
### Approximation Bayésienne par Grille

Source : [hammerdirt](https://hammerdirt-analyst.github.io/feb_2024/titlepage.html)

application : [solid-waste-team](https://hammerdirt-analyst.github.io/solid-waste-team/grid_approximation.html)

prior : Résultats de la campagne d'échantillonnage précédente

Cas d'utilisation : Cette méthode est une approche manuelle de l'inférence Bayésienne. Elle est particulièrement utile lorsque vous souhaitez incorporer des croyances antérieures et mettre à jour ces croyances avec des données observées.

Mise en œuvre : Implique la définition d'une grille de valeurs de paramètres et le calcul de la vraisemblance des données observées à chaque point de cette grille. En multipliant par la probabilité a priori et en normalisant, on obtient la distribution a posteriori. Cela peut être fait pour chaque condition séparément ou pour toutes les conditions ensemble, bien que cela soit plus intensif en termes de calcul.



::::{tab-set}

:::{tab-item} Toutes les conditions
{glue}`toutes-gapprox`
:::

:::{tab-item} Graviers
{glue}`graviers-gapprox`
:::

:::{tab-item} Sables
{glue}`sables-gapprox`

:::

:::{tab-item} Ville et haute Fréquentation
{glue}`v-hf-gapprox`

:::

:::{tab-item} Campagne et haute fréquentation
{glue}`cam-hf-gapprox`

:::

:::{tab-item} Campagne et parking <= 500 m
{glue}`c-dist1-gapprox`

:::

:::{tab-item} Ville et parking <= 500 m
{glue}`v-dist1-gapprox`

:::



:::{tab-item} Résultats
:selected:

````{grid} 1 2 2 2

```{grid-item}
{glue}`q-tous-b`
```

```{grid-item}

Prédictions : Fournit une distribution de valeurs possibles de pcs/m, offrant une idée de la fourchette et de l'incertitude des prédictions. Particulièrement utile lorsque la prise de décision nécessite de comprendre l'incertitude ou la variabilité des prédictions.

```

```{grid-item}
{glue}`q-gravier-b`
```

```{grid-item}
{glue}`q-sable-b`
```

```{grid-item}
{glue}`q-v-hf-b`
```

```{grid-item}
{glue}`q-cam-hf-b`
```

```{grid-item}
{glue}`q-c-dist1-b`
```

```{grid-item}
{glue}`q-v-dist1-b`
```

````
:::

::::
    

In [56]:
# def create_bins(predictions, bin_width=0.2):
#     """
#     Create bins from the predictions with a specified width.

#     :param predictions: List or array of prediction values.
#     :param bin_width: Width of each bin. Default is 0.2.
#     :return: A tuple (bins, bin_counts).
#         bins: The edges of the bins.
#         bin_counts: The count of predictions in each bin.
#     """
#     # Determine the range for the bins
#     max_prediction = max(predictions)
#     bins = np.arange(0, max_prediction + bin_width, bin_width)

#     # Count the number of predictions in each bin
#     bin_counts, _ = np.histogram(predictions, bins=bins)

#     return bins, bin_counts

# def calculate_bin_probabilities(bin_counts):
#     """
#     Calculate the probability for each bin.

#     :param bin_counts: The count of predictions in each bin.
#     :return: List of probabilities for each bin.
#     """
#     total_predictions = sum(bin_counts)
#     bin_probabilities = bin_counts / total_predictions
#     return bin_probabilities

# def sample_multinomial(n_samples, bin_probabilities):
#     """
#     Sample from a multinomial distribution using numpy.

#     :param n_samples: Number of samples to draw.
#     :param bin_probabilities: The probabilities of each outcome.
#     :return: Array of counts for each outcome.
#     """
#     # The number of outcomes is the length of bin_probabilities
#     n_outcomes = len(bin_probabilities)

#     # Draw samples
#     samples = np.random.multinomial(n_samples, bin_probabilities, size=1)
#     return samples

# bins, bin_counts = create_bins(predictions_flatxi)
# replace = list(set(bin_counts))[1]
# bin_counts[bin_counts == 0] = replace
# bin_probs = calculate_bin_probabilities(bin_counts)
# pred_samps = sample_multinomial(10000, bin_probs)
# bin_centers = (bins[:-1] + bins[1:]) / 2  # Calculate bin centers
# multinomial_pcs_m = []

# for bin_center, count in zip(bin_centers, pred_samps[0]):
#     multinomial_pcs_m.extend([bin_center] * count)

## Inventaire Plastock.

In [57]:
t = rc.translate_for_display(plastock_report.inventory, amap=language_maps['fr'], lan='fr')
t['objet'] = t.code.apply(lambda x: codes.loc[x, 'fr'])
t = t[[t.columns[0], t.columns[-1], *t.columns[1:-1]]]
t.set_index(['code', 'objet'], inplace=True)
t.index.name = None
t.style.set_table_styles(conf_.table_css_styles).format(**conf_.format_kwargs)

,,quantité,% du total,pcs/m,taux d'échec
code,objet,,,,
Gfrags,"Fragments de plastique: G80, G79, G78, G75",11'221,"0,41","0,72","0,97"
G27,Mégots et filtres à cigarettes,3'089,"0,11","0,15","0,79"
G30,"Emballages de bonbons, de snacks",2'080,"0,08","0,15","0,74"
G106,Fragments de plastique angulaires <5mm,1'926,"0,07","0,00","0,41"
G112,Pellets industriels (GPI),1'526,"0,06","0,00","0,36"
Gfoams,"Fragments de polystyrène expansé: G76, G81, G82, G83",1'399,"0,05","0,07","0,72"
Gcaps,"Couvercles en plastique bouteille: G21, G22, G23, G24",1'070,"0,04","0,04","0,65"
G95,Coton-tige,1'040,"0,04","0,03","0,54"
G74,Mousse de plastique pour l'isolation thermique,406,"0,01","0,00","0,38"


In [58]:
%watermark --iversions -b -r

Git repo: https://github.com/hammerdirt-analyst/plastock.git

Git branch: main

seaborn   : 0.12.2
matplotlib: 3.7.1
pandas    : 2.0.0
numpy     : 1.24.2

